In [1]:
from sqlalchemy import create_engine, Table, MetaData, insert, text, inspect
from sqlalchemy.orm import sessionmaker
import pandas as pd
import json

In [2]:
DATABASE_URL = f"mysql+pymysql://root:@localhost/csvdata01_02102024"
engine = create_engine(DATABASE_URL)
metadata = MetaData()
paximum_table = Table('juniper', metadata, autoload_with=engine)

In [3]:
try:
    with engine.connect() as connection:
        result = connection.execute(text("SELECT DATABASE()"))
        db_name = result.fetchone()[0]
        print(f"Connected to database: {db_name}")
except Exception as e:
    print(f"Error: {e}")


Connected to database: csvdata01_02102024


In [4]:
try:
    inspect_table = inspect(engine)
    tables = inspect_table.get_table_names()
    if tables:
        for table in tables:
            print(table)

except Exception as e:
    print(f"Error {e}")

hotels_info
hotels_info_with_gidestination_code
info_01
innova_hotels_main
juniper
my_table
paximum
ratehawk
z_ocas_member_data


In [13]:
import requests
import xmltodict
import json

url = "https://xml-uat.bookingengine.es/WebService/jp/operations/staticdatatransactions.asmx"

payload = "<soapenv:Envelope xmlns:soapenv=\"http://schemas.xmlsoap.org/soap/envelope/\" xmlns=\"http://www.juniper.es/webservice/2007/\">\n   <soapenv:Header/>\n   <soapenv:Body>\n      <HotelPortfolio>\n         <HotelPortfolioRQ Version=\"1.1\" Language=\"en\" Page=\"2\" RecordsPerPage=\"5\">\n            <Login Password=\"^wvW6#[49J/G\" Email=\"TestXMLNofshonTours\"/>\n         </HotelPortfolioRQ>\n      </HotelPortfolio>\n   </soapenv:Body>\n</soapenv:Envelope>"
headers = {
  'Content-Type': 'text/xml;charset=UTF-8',
  'SOAPAction': '"http://www.juniper.es/webservice/2007/HotelPortfolio"',
  'Cookie': 'StatC=8l+HAPXk1NeIsJR8qq/ISw==; StatP=lh5UxVPejVDjU3b27uc0fygTysSaay6K5gfbVxJWTYZM4f+TafxAFFOXsj/1uQ3n; idioma=en'
}

response = requests.request("POST", url, headers=headers, data=payload)

data_dict = xmltodict.parse(response.text)
print(data_dict)


{'soap:Envelope': {'@xmlns:soap': 'http://schemas.xmlsoap.org/soap/envelope/', '@xmlns:xsi': 'http://www.w3.org/2001/XMLSchema-instance', '@xmlns:xsd': 'http://www.w3.org/2001/XMLSchema', 'soap:Body': {'HotelPortfolioResponse': {'@xmlns': 'http://www.juniper.es/webservice/2007/', 'HotelPortfolioRS': {'@Url': 'http://xml-uat.bookingengine.es', '@TimeStamp': '2024-11-23T11:34:20.3714381+01:00', '@IntCode': 'FB5poqs4qMHFVbzEws9wODDtnSZiaOfoMhX3sI/X0WY=', 'HotelPortfolio': {'@Page': '2', '@TotalPages': '1952', '@TotalRecords': '9757', '@LastUpdated': '2024-09-05T20:00:59.153', 'Hotel': [{'@xsi:type': 'JP_HotelInfoPortfolio', '@JPCode': 'JP060468', '@HasSynonyms': 'false', 'Name': 'Embat', 'Zone': {'@JPDCode': 'JPD000003', '@Code': '1', 'Name': 'Can Pastilla|Palma de Mallorca|Majorca|Spain'}, 'Address': 'Carrer Nanses, 7, 07610 Can Pastilla, Illes Balears, Spain', 'ZipCode': '07610', 'Latitude': '39.5343124', 'Longitude': '2.7144459', 'HotelCategory': {'@Type': '2Est', '@Code': '2', '#text'

In [14]:
# Extract hotel data
hotels = []
hotel_list = data_dict.get("soap:Envelope", {}).get("soap:Body", {}).get(
    "HotelPortfolioResponse", {}
).get("HotelPortfolioRS", {}).get("HotelPortfolio", {}).get("Hotel", [])

# print(hotel_list)

if not hotel_list:
    print("No hotels found in the response.")
    exit()

# Process and insert hotel data into the database
with engine.connect() as conn:
    for hotel in hotel_list:
        hotel_data = {
            "HotelName": hotel.get("Name"),
            "HotelId": hotel.get("@JPCode"),
            "ZipCode": hotel.get("ZipCode"),
            "Latitude": hotel.get("Latitude"),
            "Longitude": hotel.get("Longitude"),
        }

        # Insert into the database
        try:
            stmt = insert(juniper_table).values(hotel_data)
            conn.execute(stmt)
            conn.commit()  # Save the changes immediately
            print(f"Inserted: {hotel_data}")
        except Exception as e:
            print(f"Error inserting {hotel_data}: {e}")

[{'@xsi:type': 'JP_HotelInfoPortfolio', '@JPCode': 'JP060468', '@HasSynonyms': 'false', 'Name': 'Embat', 'Zone': {'@JPDCode': 'JPD000003', '@Code': '1', 'Name': 'Can Pastilla|Palma de Mallorca|Majorca|Spain'}, 'Address': 'Carrer Nanses, 7, 07610 Can Pastilla, Illes Balears, Spain', 'ZipCode': '07610', 'Latitude': '39.5343124', 'Longitude': '2.7144459', 'HotelCategory': {'@Type': '2Est', '@Code': '2', '#text': '2 Stars'}, 'City': {'@Id': '15011', '@JPDCode': 'JPD054557', '#text': 'Palma de Mallorca'}}, {'@xsi:type': 'JP_HotelInfoPortfolio', '@JPCode': 'JP060489', '@HasSynonyms': 'false', 'Name': 'Isla del Sol Apartments', 'Zone': {'@JPDCode': 'JPD000003', '@Code': '1', 'Name': 'Can Pastilla|Palma de Mallorca|Majorca|Spain'}, 'Address': 'Carrer dels Pins, 5, 07610 Can Pastilla, Illes Balears, Spain', 'ZipCode': '07610', 'Latitude': '39.5375776', 'Longitude': '2.7142996', 'HotelCategory': {'@Code': '8'}, 'City': {'@Id': '15011', '@JPDCode': 'JPD054557', '#text': 'Palma de Mallorca'}}, {'@

In [15]:
import requests
import xmltodict
import json

url = "https://xml-uat.bookingengine.es/WebService/jp/operations/staticdatatransactions.asmx"

payload = "<soapenv:Envelope xmlns:soapenv=\"http://schemas.xmlsoap.org/soap/envelope/\" xmlns=\"http://www.juniper.es/webservice/2007/\">\n    <soapenv:Header/>\n    <soapenv:Body>\n        <HotelContent>\n            <HotelContentRQ Version=\"1\" Language=\"en\">\n                <Login Password=\"^wvW6#[49J/G\" Email=\"TestXMLNofshonTours\"/>\n                <HotelContentList>\n                    <Hotel Code=\"JP278878\"/>\n                </HotelContentList>\n            </HotelContentRQ>\n        </HotelContent>\n    </soapenv:Body>\n</soapenv:Envelope>"
headers = {
  'Content-Type': 'text/xml;charset=UTF-8',
  'SOAPAction': '"http://www.juniper.es/webservice/2007/HotelContent"',
  'Cookie': 'StatC=8l+HAPXk1NeIsJR8qq/ISw==; StatP=UAvxTZg5/KSb5xjepzejGq+uRWuHJ5ucWURdLMNyct+5jbHBZLi9nj/RVgL8/LVZ; idioma=en'
}

response = requests.request("POST", url, headers=headers, data=payload)

# print(response.text)
data_dict = xmltodict.parse(response.text)
print(data_dict)


{'soap:Envelope': {'@xmlns:soap': 'http://schemas.xmlsoap.org/soap/envelope/', '@xmlns:xsi': 'http://www.w3.org/2001/XMLSchema-instance', '@xmlns:xsd': 'http://www.w3.org/2001/XMLSchema', 'soap:Body': {'HotelContentResponse': {'@xmlns': 'http://www.juniper.es/webservice/2007/', 'ContentRS': {'@Url': 'http://xml-uat.bookingengine.es', '@TimeStamp': '2024-11-23T13:22:15.2425113+01:00', '@IntCode': 'iRuY0WaaloeCOxVJtaKWUnXIM/qOrUFMDjOp5qV50hI=', 'Contents': {'HotelContent': {'@Code': 'JP278878', '@JPCode': 'JP278878', 'HotelName': 'The Peninsula Chittagong', 'Zone': {'@JPDCode': 'JPD174295', '@Code': '132510', 'Name': 'Chittagong, Chittagong, Bangladesh'}, 'HotelChain': {'Name': '0|No chain|'}, 'HotelCategory': {'@Type': '4est', '#text': '4 Stars'}, 'Address': {'Address': 'Bulbul Center, 486/B CDA Ave, Chittagong 4000, Bangladesh', 'PostalCode': '4000', 'Latitude': '22.3577436', 'Longitude': '91.8218918'}, 'Images': {'Image': [{'@Type': 'BIG', 'FileName': 'https://pix8.agoda.net/hotelImag

In [ ]:
{'soap:Envelope': {'@xmlns:soap': 'http://schemas.xmlsoap.org/soap/envelope/', '@xmlns:xsi': 'http://www.w3.org/2001/XMLSchema-instance', '@xmlns:xsd': 'http://www.w3.org/2001/XMLSchema', 'soap:Body': {'HotelContentResponse': {'@xmlns': 'http://www.juniper.es/webservice/2007/', 'ContentRS': {'@Url': 'http://xml-uat.bookingengine.es', '@TimeStamp': '2024-11-23T13:22:15.2425113+01:00', '@IntCode': 'iRuY0WaaloeCOxVJtaKWUnXIM/qOrUFMDjOp5qV50hI=', 'Contents': {'HotelContent': {'@Code': 'JP278878', '@JPCode': 'JP278878', 'HotelName': 'The Peninsula Chittagong', 'Zone': {'@JPDCode': 'JPD174295', '@Code': '132510', 'Name': 'Chittagong, Chittagong, Bangladesh'}, 'HotelChain': {'Name': '0|No chain|'}, 'HotelCategory': {'@Type': '4est', '#text': '4 Stars'}, 'Address': {'Address': 'Bulbul Center, 486/B CDA Ave, Chittagong 4000, Bangladesh', 'PostalCode': '4000', 'Latitude': '22.3577436', 'Longitude': '91.8218918'}, 'Images': {'Image': [{'@Type': 'BIG', 'FileName': 'https://pix8.agoda.net/hotelImages/295517/-1/a563911377c758dd22342c3fad66d229.jpg?ce=0&s=2000x', 'Title': '295517-127029872', 'Description': 'Entrance'}, {'@Type': 'THB', 'FileName': 'https://pix8.agoda.net/hotelImages/295517/-1/a563911377c758dd22342c3fad66d229.jpg?ce=0&s=350x', 'Title': '295517-127029872'}, {'@Type': 'BIG', 'FileName': 'https://pix8.agoda.net/hotelImages/295517/-1/40e114843553a4029fc89345c6c74a84.jpg?ce=0&s=2000x', 'Title': '295517-127029775', 'Description': 'Swimming pool'}, {'@Type': 'THB', 'FileName': 'https://pix8.agoda.net/hotelImages/295517/-1/40e114843553a4029fc89345c6c74a84.jpg?ce=0&s=350x', 'Title': '295517-127029775'}, {'@Type': 'BIG', 'FileName': 'https://pix8.agoda.net/hotelImages/295517/-1/c9269bda1f470e23248402a3c48399d7.jpg?ce=0&s=2000x', 'Title': '295517-127029687', 'Description': 'Restaurant'}, {'@Type': 'THB', 'FileName': 'https://pix8.agoda.net/hotelImages/295517/-1/c9269bda1f470e23248402a3c48399d7.jpg?ce=0&s=350x', 'Title': '295517-127029687'}, {'@Type': 'BIG', 'FileName': 'https://pix8.agoda.net/hotelImages/295517/-1/44f5e577e96702a8f88a6c04b51ac65d.jpg?ce=0&s=2000x', 'Title': '295517-127029845', 'Description': 'Lobby'}, {'@Type': 'THB', 'FileName': 'https://pix8.agoda.net/hotelImages/295517/-1/44f5e577e96702a8f88a6c04b51ac65d.jpg?ce=0&s=350x', 'Title': '295517-127029845'}, {'@Type': 'BIG', 'FileName': 'https://q-xx.bstatic.com/xdata/images/hotel/max500/575347609.jpg?k=ff1568e533c41b3f44c0ed7550c069064dc7fd6d3b470f33d386a24fafabb7ad&o=&s=2000x', 'Title': '295517-575452352', 'Description': 'Spa'}, {'@Type': 'THB', 'FileName': 'https://q-xx.bstatic.com/xdata/images/hotel/max500/575347609.jpg?k=ff1568e533c41b3f44c0ed7550c069064dc7fd6d3b470f33d386a24fafabb7ad&o=&s=350x', 'Title': '295517-575452352'}, {'@Type': 'BIG', 'FileName': 'https://q-xx.bstatic.com/xdata/images/hotel/max500/575343964.jpg?k=439bebb3a2d6af24e52058b9aa7ebbe8e219f51255e969c5db352cbd1ec409e4&o=&s=2000x', 'Title': '295517-575452350', 'Description': 'Facilities'}, {'@Type': 'THB', 'FileName': 'https://q-xx.bstatic.com/xdata/images/hotel/max500/575343964.jpg?k=439bebb3a2d6af24e52058b9aa7ebbe8e219f51255e969c5db352cbd1ec409e4&o=&s=350x', 'Title': '295517-575452350'}, {'@Type': 'BIG', 'FileName': 'https://pix8.agoda.net/hotelImages/295517/-1/61b462124920752bfe4697337e86f25b.jpg?ce=0&s=2000x', 'Title': '295517-127029824', 'Description': 'Swimming pool'}, {'@Type': 'THB', 'FileName': 'https://pix8.agoda.net/hotelImages/295517/-1/61b462124920752bfe4697337e86f25b.jpg?ce=0&s=350x', 'Title': '295517-127029824'}, {'@Type': 'BIG', 'FileName': 'https://q-xx.bstatic.com/xdata/images/hotel/max500/575341867.jpg?k=fbec7d066d1a56a47adb6487d4f13871a6586f86d57f8fed53d9b9d97deb76d6&o=&s=2000x', 'Title': '295517-575452361', 'Description': 'Swimming pool'}, {'@Type': 'THB', 'FileName': 'https://q-xx.bstatic.com/xdata/images/hotel/max500/575341867.jpg?k=fbec7d066d1a56a47adb6487d4f13871a6586f86d57f8fed53d9b9d97deb76d6&o=&s=350x', 'Title': '295517-575452361'}, {'@Type': 'BIG', 'FileName': 'https://q-xx.bstatic.com/xdata/images/hotel/max500/575343973.jpg?k=657b5872421947580ee2f9f1a6b79a07e1fca119c4f7d9a761bac16de2b1e4f2&o=&s=2000x', 'Title': '295517-575452353', 'Description': 'Swimming pool'}, {'@Type': 'THB', 'FileName': 'https://q-xx.bstatic.com/xdata/images/hotel/max500/575343973.jpg?k=657b5872421947580ee2f9f1a6b79a07e1fca119c4f7d9a761bac16de2b1e4f2&o=&s=350x', 'Title': '295517-575452353'}, {'@Type': 'BIG', 'FileName': 'https://pix8.agoda.net/hotelImages/295517/-1/7c6f31d8c3b2e3110a394ba40152b90c.jpg?ce=0&s=2000x', 'Title': '295517-127029822', 'Description': 'Bar'}, {'@Type': 'THB', 'FileName': 'https://pix8.agoda.net/hotelImages/295517/-1/7c6f31d8c3b2e3110a394ba40152b90c.jpg?ce=0&s=350x', 'Title': '295517-127029822'}, {'@Type': 'BIG', 'FileName': 'https://q-xx.bstatic.com/xdata/images/hotel/max500/575338695.jpg?k=dbea7b74cca4b786a512f9e40276d8a9d3019e342210bf62bca019cc0abc7ea2&o=&s=2000x', 'Title': '295517-575452345', 'Description': 'View'}, {'@Type': 'THB', 'FileName': 'https://q-xx.bstatic.com/xdata/images/hotel/max500/575338695.jpg?k=dbea7b74cca4b786a512f9e40276d8a9d3019e342210bf62bca019cc0abc7ea2&o=&s=350x', 'Title': '295517-575452345'}, {'@Type': 'BIG', 'FileName': 'https://pix8.agoda.net/property/60337436/0/adee9d955c861892036f78976228f481.jpeg?ce=0&s=2000x', 'Title': '295517-800457732'}, {'@Type': 'THB', 'FileName': 'https://pix8.agoda.net/property/60337436/0/adee9d955c861892036f78976228f481.jpeg?ce=0&s=350x', 'Title': '295517-800457732'}, {'@Type': 'BIG', 'FileName': 'https://pix8.agoda.net/hotelImages/295517/-1/c75872ef037d8fae0c41bdfeb219e49a.jpg?ce=0&s=2000x', 'Title': '295517-127029873', 'Description': 'Entrance'}, {'@Type': 'THB', 'FileName': 'https://pix8.agoda.net/hotelImages/295517/-1/c75872ef037d8fae0c41bdfeb219e49a.jpg?ce=0&s=350x', 'Title': '295517-127029873'}, {'@Type': 'BIG', 'FileName': 'https://q-xx.bstatic.com/xdata/images/hotel/max500/575343978.jpg?k=94696be9412e83f2e8443182a37ee4963142d71940b7474e347c23aa880f6e8b&o=&s=2000x', 'Title': '295517-575452369', 'Description': 'Bar/lounge'}, {'@Type': 'THB', 'FileName': 'https://q-xx.bstatic.com/xdata/images/hotel/max500/575343978.jpg?k=94696be9412e83f2e8443182a37ee4963142d71940b7474e347c23aa880f6e8b&o=&s=350x', 'Title': '295517-575452369'}, {'@Type': 'BIG', 'FileName': 'https://pix8.agoda.net/hotelImages/295517/-1/6df052e3ce79f368291c1af1d52b22c6.jpg?ce=0&s=2000x', 'Title': '295517-127029839', 'Description': 'Lobby'}, {'@Type': 'THB', 'FileName': 'https://pix8.agoda.net/hotelImages/295517/-1/6df052e3ce79f368291c1af1d52b22c6.jpg?ce=0&s=350x', 'Title': '295517-127029839'}, {'@Type': 'BIG', 'FileName': 'https://pix8.agoda.net/hotelImages/295517/-1/1d3abc22379b1f13c3af9a941e0f5835.jpg?ce=0&s=2000x', 'Title': '295517-127029833', 'Description': 'Restaurant'}, {'@Type': 'THB', 'FileName': 'https://pix8.agoda.net/hotelImages/295517/-1/1d3abc22379b1f13c3af9a941e0f5835.jpg?ce=0&s=350x', 'Title': '295517-127029833'}, {'@Type': 'BIG', 'FileName': 'https://pix8.agoda.net/property/30936078/0/8248f2751d90848c8a7ac6f81b090270.jpg?ca=28&ce=0&s=2000x', 'Title': '295517-418915004'}, {'@Type': 'THB', 'FileName': 'https://pix8.agoda.net/property/30936078/0/8248f2751d90848c8a7ac6f81b090270.jpg?ca=28&ce=0&s=350x', 'Title': '295517-418915004'}, {'@Type': 'BIG', 'FileName': 'https://pix8.agoda.net/hotelImages/295517/-1/9de4166ca6ca42b0d064432e7344db49.jpg?ce=0&s=2000x', 'Title': '295517-127029769', 'Description': 'Restaurant'}, {'@Type': 'THB', 'FileName': 'https://pix8.agoda.net/hotelImages/295517/-1/9de4166ca6ca42b0d064432e7344db49.jpg?ce=0&s=350x', 'Title': '295517-127029769'}, {'@Type': 'BIG', 'FileName': 'https://q-xx.bstatic.com/xdata/images/hotel/max500/575340531.jpg?k=a9fa031a2e27cc2810bfa0fa4e532d352562779071fb83e9301ced8b508c779a&o=&s=2000x', 'Title': '295517-575452346', 'Description': 'Restaurant'}, {'@Type': 'THB', 'FileName': 'https://q-xx.bstatic.com/xdata/images/hotel/max500/575340531.jpg?k=a9fa031a2e27cc2810bfa0fa4e532d352562779071fb83e9301ced8b508c779a&o=&s=350x', 'Title': '295517-575452346'}, {'@Type': 'BIG', 'FileName': 'https://q-xx.bstatic.com/xdata/images/hotel/max500/575346978.jpg?k=be759eddc191724a4185c9bbc223b70957782c2eef53306e6f8f9613585f1a39&o=&s=2000x', 'Title': '295517-575452368', 'Description': 'Restaurant'}, {'@Type': 'THB', 'FileName': 'https://q-xx.bstatic.com/xdata/images/hotel/max500/575346978.jpg?k=be759eddc191724a4185c9bbc223b70957782c2eef53306e6f8f9613585f1a39&o=&s=350x', 'Title': '295517-575452368'}, {'@Type': 'BIG', 'FileName': 'https://q-xx.bstatic.com/xdata/images/hotel/max500/575345234.jpg?k=fac1cbcfd1d66298a023cff938c4852c58ec5a1262036ca8c803c50c0ef7a72a&o=&s=2000x', 'Title': '295517-575452451'}, {'@Type': 'THB', 'FileName': 'https://q-xx.bstatic.com/xdata/images/hotel/max500/575345234.jpg?k=fac1cbcfd1d66298a023cff938c4852c58ec5a1262036ca8c803c50c0ef7a72a&o=&s=350x', 'Title': '295517-575452451'}, {'@Type': 'BIG', 'FileName': 'https://q-xx.bstatic.com/xdata/images/hotel/max500/295997263.jpg?k=1b5321bacbe16e0a5025d7bc84d64b8be6a74b2e2e5289f52f75ffa41f233809&o=&s=2000x', 'Title': '295517-254803639'}, {'@Type': 'THB', 'FileName': 'https://q-xx.bstatic.com/xdata/images/hotel/max500/295997263.jpg?k=1b5321bacbe16e0a5025d7bc84d64b8be6a74b2e2e5289f52f75ffa41f233809&o=&s=350x', 'Title': '295517-254803639'}, {'@Type': 'BIG', 'FileName': 'https://q-xx.bstatic.com/xdata/images/hotel/max500/575805614.jpg?k=2a2731717d88e584751a12f12ec000b2bcc555e0dd746e9035cf92c09e080ac9&o=&s=2000x', 'Title': '295517-577263467'}, {'@Type': 'THB', 'FileName': 'https://q-xx.bstatic.com/xdata/images/hotel/max500/575805614.jpg?k=2a2731717d88e584751a12f12ec000b2bcc555e0dd746e9035cf92c09e080ac9&o=&s=350x', 'Title': '295517-577263467'}]}, 'Descriptions': {'Description': [{'@Type': 'SHT', '#text': 'Great care is taken to ensure guests experience comfort through top-notch services and amenities.&lt;br&gt; Re'}, {'@Type': 'LNG', '#text': 'DESCLARGA¬Great care is taken to ensure guests experience comfort through top-notch services and amenities.&lt;br&gt; Remain linked during your visit by utilizing the complimentary internet access available.&lt;br&gt;Prior to your check-in date, you can arrange airport transportation services, guaranteeing a seamless and efficient experience for both arrival and departure.&lt;br&gt; Parking is accessible, supplied by the hotel for guests who arrive by car.&lt;br&gt;In limited designated zones, smoking is exclusively permitted.&lt;br&gt; Each morning at The Peninsula Chittagong Hotel, a scrumptious, homemade breakfast kick-starts the day.&lt;br&gt; Begin your holiday mornings right with your essential cup of coffee, offered daily at the cafe on-site.&lt;br&gt;During your visit, indulge in a range of delightful culinary choices at hotel to enhance your experience.&lt;br&gt;Indulge in the numerous pursuits available at The Peninsula Chittagong Hotel.&lt;br&gt; During your stay, the hotel provides direct access to a beach, ensuring you remain near the sea throughout your visit.&lt;br&gt;Begin your holiday perfectly by taking a plunge into the swimming pool.&lt;br&gt; At the hotel, enjoy a laid-back beverage experience by the poolside bar, sipping on a soothing cocktail.&lt;br&gt;Eliminate those holiday calories by stopping by hotel and making use of their well-equipped exercise amenities.&lt;br&gt;'}, {'@Type': 'ROO', '#text': 'Extra Bed Policy: &lt;strong&gt;Children and extra beds&lt;/strong&gt;&lt;p&gt;Minimum age of guests is: 0 year(s). &lt;br&gt;Children from 2 to 13 year(s) [inclusive]. If you need an extra bed, it will incur an additional charge.&lt;br&gt;Guests 0 years and older are considered as adults must use an extra bed which will incur an additional charge.&lt;br&gt;'}, {'@Type': 'OTH', '#text': 'Remark: None'}]}, 'Features': {'Feature': [{'@Type': 'HOT', '@ExtraFee': 'false', '#text': 'English'}, {'@Type': 'HOT', '@ExtraFee': 'false', '#text': 'Arabic'}, {'@Type': 'HOT', '@ExtraFee': 'false', '#text': 'Hindi'}, {'@Type': 'HOT', '@ExtraFee': 'false', '#text': 'Wheelchair accessible'}, {'@Type': 'HOT', '@ExtraFee': 'false', '#text': 'Free Wi-Fi in all rooms!'}, {'@Type': 'HOT', '@ExtraFee': 'false', '#text': 'Internet'}, {'@Type': 'HOT', '@ExtraFee': 'false', '#text': 'Internet services'}, {'@Type': 'HOT', '@ExtraFee': 'false', '#text': 'Wi-Fi in public areas'}, {'@Type': 'HOT', '@ExtraFee': 'false', '#text': 'Billiards'}, {'@Type': 'HOT', '@ExtraFee': 'false', '#text': 'Dart board'}, {'@Type': 'HOT', '@ExtraFee': 'false', '#text': 'Fitness center'}, {'@Type': 'HOT', '@ExtraFee': 'false', '#text': 'Game room'}, {'@Type': 'HOT', '@ExtraFee': 'false', '#text': 'Garden'}, {'@Type': 'HOT', '@ExtraFee': 'false', '#text': 'Hot tub'}, {'@Type': 'HOT', '@ExtraFee': 'false', '#text': 'Massage'}, {'@Type': 'HOT', '@ExtraFee': 'false', '#text': 'Nightclub'}, {'@Type': 'HOT', '@ExtraFee': 'false', '#text': 'Sauna'}, {'@Type': 'HOT', '@ExtraFee': 'false', '#text': 'Spa'}, {'@Type': 'HOT', '@ExtraFee': 'false', '#text': 'Spa/sauna'}, {'@Type': 'HOT', '@ExtraFee': 'false', '#text': 'Steamroom'}, {'@Type': 'HOT', '@ExtraFee': 'false', '#text': 'Swimming pool [indoor]'}, {'@Type': 'HOT', '@ExtraFee': 'false', '#text': 'Swimming pool [outdoor]'}, {'@Type': 'HOT', '@ExtraFee': 'false', '#text': 'Table tennis'}, {'@Type': 'HOT', '@ExtraFee': 'false', '#text': 'Tours'}, {'@Type': 'HOT', '@ExtraFee': 'false', '#text': 'Anti-viral cleaning products'}, {'@Type': 'HOT', '@ExtraFee': 'false', '#text': 'Body thermometer'}, {'@Type': 'HOT', '@ExtraFee': 'false', '#text': 'Breakfast in room'}, {'@Type': 'HOT', '@ExtraFee': 'false', '#text': 'Breakfast takeaway service'}, {'@Type': 'HOT', '@ExtraFee': 'false', '#text': 'Cashless payment service'}, {'@Type': 'HOT', '@ExtraFee': 'false', '#text': 'Doctor/nurse on call'}, {'@Type': 'HOT', '@ExtraFee': 'false', '#text': 'First aid kit'}, {'@Type': 'HOT', '@ExtraFee': 'false', '#text': 'Free face masks'}, {'@Type': 'HOT', '@ExtraFee': 'false', '#text': 'Guest rooms seal after sanitization'}, {'@Type': 'HOT', '@ExtraFee': 'false', '#text': 'Hand sanitizer'}, {'@Type': 'HOT', '@ExtraFee': 'false', '#text': 'Hot water linen and laundry washing'}, {'@Type': 'HOT', '@ExtraFee': 'false', '#text': 'Individually-wrapped food options'}, {'@Type': 'HOT', '@ExtraFee': 'false', '#text': 'Physical distancing of at least 1 meter'}, {'@Type': 'HOT', '@ExtraFee': 'false', '#text': 'Protective screens in common areas'}, {'@Type': 'HOT', '@ExtraFee': 'false', '#text': 'Room sanitization opt-out available'}, {'@Type': 'HOT', '@ExtraFee': 'false', '#text': 'Rooms sanitized between stays'}, {'@Type': 'HOT', '@ExtraFee': 'false', '#text': 'Safe dining setup'}, {'@Type': 'HOT', '@ExtraFee': 'false', '#text': 'Sanitized kitchen and tableware items'}, {'@Type': 'HOT', '@ExtraFee': 'false', '#text': 'Shared stationery removed'}, {'@Type': 'HOT', '@ExtraFee': 'false', '#text': 'Staff trained in safety protocol'}, {'@Type': 'HOT', '@ExtraFee': 'false', '#text': 'Temperature check for guests and staff'}, {'@Type': 'HOT', '@ExtraFee': 'false', '#text': 'Alternative meal arrangement'}, {'@Type': 'HOT', '@ExtraFee': 'false', '#text': 'Bar'}, {'@Type': 'HOT', '@ExtraFee': 'false', '#text': 'BBQ facilities'}, {'@Type': 'HOT', '@ExtraFee': 'false', '#text': 'Breakfast [buffet]'}, {'@Type': 'HOT', '@ExtraFee': 'false', '#text': 'Breakfast [continental]'}, {'@Type': 'HOT', '@ExtraFee': 'false', '#text': 'Coffee shop'}, {'@Type': 'HOT', '@ExtraFee': 'false', '#text': 'Poolside bar'}, {'@Type': 'HOT', '@ExtraFee': 'false', '#text': 'Restaurants'}, {'@Type': 'HOT', '@ExtraFee': 'false', '#text': 'Room service [24-hour]'}, {'@Type': 'HOT', '@ExtraFee': 'false', '#text': 'Snack bar'}, {'@Type': 'HOT', '@ExtraFee': 'false', '#text': 'Air conditioning in public area'}, {'@Type': 'HOT', '@ExtraFee': 'false', '#text': 'Concierge'}, {'@Type': 'HOT', '@ExtraFee': 'false', '#text': 'Contactless check-in/out'}, {'@Type': 'HOT', '@ExtraFee': 'false', '#text': 'Convenience store'}, {'@Type': 'HOT', '@ExtraFee': 'false', '#text': 'Currency exchange'}, {'@Type': 'HOT', '@ExtraFee': 'false', '#text': 'Daily housekeeping'}, {'@Type': 'HOT', '@ExtraFee': 'false', '#text': 'Doorman'}, {'@Type': 'HOT', '@ExtraFee': 'false', '#text': 'Dry cleaning'}, {'@Type': 'HOT', '@ExtraFee': 'false', '#text': 'Elevator'}, {'@Type': 'HOT', '@ExtraFee': 'false', '#text': 'Facilities for disabled guests'}, {'@Type': 'HOT', '@ExtraFee': 'false', '#text': 'Food delivery'}, {'@Type': 'HOT', '@ExtraFee': 'false', '#text': 'Gift/souvenir shop'}, {'@Type': 'HOT', '@ExtraFee': 'false', '#text': 'Grooming service'}, {'@Type': 'HOT', '@ExtraFee': 'false', '#text': 'Invoice provided'}, {'@Type': 'HOT', '@ExtraFee': 'false', '#text': 'Ironing service'}, {'@Type': 'HOT', '@ExtraFee': 'false', '#text': 'Laundry service'}, {'@Type': 'HOT', '@ExtraFee': 'false', '#text': 'Luggage storage'}, {'@Type': 'HOT', '@ExtraFee': 'false', '#text': 'Meeting/banquet facilities'}, {'@Type': 'HOT', '@ExtraFee': 'false', '#text': 'Safety deposit boxes'}, {'@Type': 'HOT', '@ExtraFee': 'false', '#text': 'Salon'}, {'@Type': 'HOT', '@ExtraFee': 'false', '#text': 'Shared lounge/TV area'}, {'@Type': 'HOT', '@ExtraFee': 'false', '#text': 'Shops'}, {'@Type': 'HOT', '@ExtraFee': 'false', '#text': 'Smoke-free property'}, {'@Type': 'HOT', '@ExtraFee': 'false', '#text': 'Smoking area'}, {'@Type': 'HOT', '@ExtraFee': 'false', '#text': 'Terrace'}, {'@Type': 'HOT', '@ExtraFee': 'false', '#text': 'Xerox/fax in business center'}, {'@Type': 'HOT', '@ExtraFee': 'false', '#text': 'Family room'}, {'@Type': 'HOT', '@ExtraFee': 'false', '#text': 'Kids meal'}, {'@Type': 'HOT', '@ExtraFee': 'false', '#text': 'Playground'}, {'@Type': 'HOT', '@ExtraFee': 'false', '#text': 'Golf course (within 3 km)'}, {'@Type': 'HOT', '@ExtraFee': 'false', '#text': 'CCTV in common areas'}, {'@Type': 'HOT', '@ExtraFee': 'false', '#text': 'CCTV outside property'}, {'@Type': 'HOT', '@ExtraFee': 'false', '#text': 'Check-in/out [express]'}, {'@Type': 'HOT', '@ExtraFee': 'false', '#text': 'Fire extinguisher'}, {'@Type': 'HOT', '@ExtraFee': 'false', '#text': 'Front desk [24-hour]'}, {'@Type': 'HOT', '@ExtraFee': 'false', '#text': 'Non-smoking rooms'}, {'@Type': 'HOT', '@ExtraFee': 'false', '#text': 'Safety/security feature'}, {'@Type': 'HOT', '@ExtraFee': 'false', '#text': 'Security [24-hour]'}, {'@Type': 'HOT', '@ExtraFee': 'false', '#text': 'Smoke alarms'}, {'@Type': 'HOT', '@ExtraFee': 'false', '#text': 'Airport transfer'}, {'@Type': 'HOT', '@ExtraFee': 'false', '#text': 'Car park [free of charge]'}, {'@Type': 'HOT', '@ExtraFee': 'false', '#text': 'Rental car'}, {'@Type': 'HOT', '@ExtraFee': 'false', '#text': 'Shuttle service'}, {'@Type': 'HOT', '@ExtraFee': 'false', '#text': 'Valet parking'}, {'@Type': 'HOT', '@ExtraFee': 'false', '#text': 'Accessible by elevator'}, {'@Type': 'HOT', '@ExtraFee': 'false', '#text': 'Adapter'}, {'@Type': 'HOT', '@ExtraFee': 'false', '#text': 'Air conditioning'}, {'@Type': 'HOT', '@ExtraFee': 'false', '#text': 'Bathtub'}, {'@Type': 'HOT', '@ExtraFee': 'false', '#text': 'Closet'}, {'@Type': 'HOT', '@ExtraFee': 'false', '#text': 'Clothes rack'}, {'@Type': 'HOT', '@ExtraFee': 'false', '#text': 'Coffee/tea maker'}, {'@Type': 'HOT', '@ExtraFee': 'false', '#text': 'Desk'}, {'@Type': 'HOT', '@ExtraFee': 'false', '#text': 'Electric kettle'}, {'@Type': 'HOT', '@ExtraFee': 'false', '#text': 'Free bottled water'}, {'@Type': 'HOT', '@ExtraFee': 'false', '#text': 'Fruits/snacks'}, {'@Type': 'HOT', '@ExtraFee': 'false', '#text': 'Hair dryer'}, {'@Type': 'HOT', '@ExtraFee': 'false', '#text': 'In-room safe box'}, {'@Type': 'HOT', '@ExtraFee': 'false', '#text': 'Ironing facilities'}, {'@Type': 'HOT', '@ExtraFee': 'false', '#text': 'Linens'}, {'@Type': 'HOT', '@ExtraFee': 'false', '#text': 'Mini bar'}, {'@Type': 'HOT', '@ExtraFee': 'false', '#text': 'Private bathroom'}, {'@Type': 'HOT', '@ExtraFee': 'false', '#text': 'Refrigerator'}, {'@Type': 'HOT', '@ExtraFee': 'false', '#text': 'Satellite/cable channels'}, {'@Type': 'HOT', '@ExtraFee': 'false', '#text': 'Shower'}, {'@Type': 'HOT', '@ExtraFee': 'false', '#text': 'Sleep comfort items'}, {'@Type': 'HOT', '@ExtraFee': 'false', '#text': 'Slippers'}, {'@Type': 'HOT', '@ExtraFee': 'false', '#text': 'Smoke detector'}, {'@Type': 'HOT', '@ExtraFee': 'false', '#text': 'Socket near the bed'}, {'@Type': 'HOT', '@ExtraFee': 'false', '#text': 'Telephone'}, {'@Type': 'HOT', '@ExtraFee': 'false', '#text': 'Toiletries'}, {'@Type': 'HOT', '@ExtraFee': 'false', '#text': 'Towels'}, {'@Type': 'HOT', '@ExtraFee': 'false', '#text': 'Trash cans'}]}, 'PointsOfInterest': {'PointOfInterest': [{'Name': None, 'Distance': '0.6 \xa0kilomètres'}, {'Name': None, 'Distance': '0.76 \xa0kilomètres'}, {'Name': None, 'Distance': '1.2 \xa0kilomètres'}, {'Name': None, 'Distance': '1.62 \xa0kilomètres'}, {'Name': None, 'Distance': '1.74 \xa0kilomètres'}, {'Name': None, 'Distance': '1.92 \xa0kilomètres'}, {'Name': None, 'Distance': '2.37 \xa0kilomètres'}, {'Name': None, 'Distance': '2.45 \xa0kilomètres'}, {'Name': None, 'Distance': '2.86 \xa0kilomètres'}, {'Name': None, 'Distance': '3.39 \xa0kilomètres'}]}, 'TimeInformation': {'CheckTime': {'@CheckIn': '02:00 PM', '@CheckOut': '12:00 PM'}}, 'HotelRooms': {'HotelRoom': [{'Name': 'Executive Suite', 'Description': 'Executive Suite - (1 king bed) - size_room: 32', 'RoomOccupancy': {'@MaxAdults': '4', '@MaxChildren': '0'}}, {'Name': 'Executive Suite', 'Description': 'Executive Suite - (1 king bed) - size_room: 32', 'RoomOccupancy': {'@MaxAdults': '4', '@MaxChildren': '0'}}, {'Name': 'Executive Suite', 'Description': 'Executive Suite - (1 king bed) - size_room: 32', 'RoomOccupancy': {'@MaxAdults': '4', '@MaxChildren': '0'}}, {'Name': 'Executive Suite', 'Description': 'Executive Suite - (1 king bed) - size_room: 32', 'RoomOccupancy': {'@MaxAdults': '4', '@MaxChildren': '0'}}, {'Name': 'Executive Suite', 'Description': 'Executive Suite - (1 king bed) - size_room: 32', 'RoomOccupancy': {'@MaxAdults': '4', '@MaxChildren': '0'}}, {'Name': 'Executive Suite', 'Description': 'Executive Suite - (1 king bed) - size_room: 32', 'RoomOccupancy': {'@MaxAdults': '4', '@MaxChildren': '0'}}, {'Name': 'Executive Suite', 'Description': 'Executive Suite - (1 king bed) - size_room: 32', 'RoomOccupancy': {'@MaxAdults': '4', '@MaxChildren': '0'}}, {'Name': 'Executive Suite', 'Description': 'Executive Suite - (1 king bed) - size_room: 32', 'RoomOccupancy': {'@MaxAdults': '4', '@MaxChildren': '0'}}, {'Name': 'Executive Suite', 'Description': 'Executive Suite - (1 king bed) - size_room: 32', 'RoomOccupancy': {'@MaxAdults': '4', '@MaxChildren': '0'}}, {'Name': 'Executive Suite', 'Description': 'Executive Suite - (1 king bed) - size_room: 32', 'RoomOccupancy': {'@MaxAdults': '4', '@MaxChildren': '0'}}, {'Name': 'Executive Suite', 'Description': 'Executive Suite - (1 king bed) - size_room: 32', 'RoomOccupancy': {'@MaxAdults': '4', '@MaxChildren': '0'}}, {'Name': 'Executive Suite', 'Description': 'Executive Suite - (1 king bed) - size_room: 32', 'RoomOccupancy': {'@MaxAdults': '4', '@MaxChildren': '0'}}, {'Name': 'Executive Suite', 'Description': 'Executive Suite - (1 king bed) - size_room: 32', 'RoomOccupancy': {'@MaxAdults': '4', '@MaxChildren': '0'}}, {'Name': 'Executive Suite', 'Description': 'Executive Suite - (1 king bed) - size_room: 32', 'RoomOccupancy': {'@MaxAdults': '4', '@MaxChildren': '0'}}, {'Name': 'Executive Suite', 'Description': 'Executive Suite - (1 king bed) - size_room: 32', 'RoomOccupancy': {'@MaxAdults': '4', '@MaxChildren': '0'}}, {'Name': 'Executive Suite', 'Description': 'Executive Suite - (1 king bed) - size_room: 32', 'RoomOccupancy': {'@MaxAdults': '4', '@MaxChildren': '0'}}, {'Name': 'Executive Suite', 'Description': 'Executive Suite - (1 king bed) - size_room: 32', 'RoomOccupancy': {'@MaxAdults': '4', '@MaxChildren': '0'}}, {'Name': 'Executive Suite', 'Description': 'Executive Suite - (1 king bed) - size_room: 32', 'RoomOccupancy': {'@MaxAdults': '4', '@MaxChildren': '0'}}, {'Name': 'Executive Suite', 'Description': 'Executive Suite - (1 king bed) - size_room: 32', 'RoomOccupancy': {'@MaxAdults': '4', '@MaxChildren': '0'}}, {'Name': 'Executive Suite', 'Description': 'Executive Suite - (1 king bed) - size_room: 32', 'RoomOccupancy': {'@MaxAdults': '4', '@MaxChildren': '0'}}, {'Name': 'Executive Suite', 'Description': 'Executive Suite - (1 king bed) - size_room: 32', 'RoomOccupancy': {'@MaxAdults': '4', '@MaxChildren': '0'}}, {'Name': 'Executive Suite', 'Description': 'Executive Suite - (1 king bed) - size_room: 32', 'RoomOccupancy': {'@MaxAdults': '4', '@MaxChildren': '0'}}, {'Name': 'Executive Suite', 'Description': 'Executive Suite - (1 king bed) - size_room: 32', 'RoomOccupancy': {'@MaxAdults': '4', '@MaxChildren': '0'}}, {'Name': 'Executive Suite', 'Description': 'Executive Suite - (1 king bed) - size_room: 32', 'RoomOccupancy': {'@MaxAdults': '4', '@MaxChildren': '0'}}, {'Name': 'Executive Suite', 'Description': 'Executive Suite - (1 king bed) - size_room: 32', 'RoomOccupancy': {'@MaxAdults': '4', '@MaxChildren': '0'}}, {'Name': 'Executive Suite', 'Description': 'Executive Suite - (1 king bed) - size_room: 32', 'RoomOccupancy': {'@MaxAdults': '4', '@MaxChildren': '0'}}, {'Name': 'Executive Suite', 'Description': 'Executive Suite - (1 king bed) - size_room: 32', 'RoomOccupancy': {'@MaxAdults': '4', '@MaxChildren': '0'}}, {'Name': 'Super Deluxe Single', 'Description': 'Super Deluxe Single - (1 single bed) - size_room: 0', 'RoomOccupancy': {'@MaxAdults': '2', '@MaxChildren': '0'}}, {'Name': 'Super Deluxe Single', 'Description': 'Super Deluxe Single - (1 single bed) - size_room: 0', 'RoomOccupancy': {'@MaxAdults': '2', '@MaxChildren': '0'}}, {'Name': 'Super Deluxe Single', 'Description': 'Super Deluxe Single - (1 single bed) - size_room: 0', 'RoomOccupancy': {'@MaxAdults': '2', '@MaxChildren': '0'}}, {'Name': 'Super Deluxe Single', 'Description': 'Super Deluxe Single - (1 single bed) - size_room: 0', 'RoomOccupancy': {'@MaxAdults': '2', '@MaxChildren': '0'}}, {'Name': 'Super Deluxe Single', 'Description': 'Super Deluxe Single - (1 single bed) - size_room: 0', 'RoomOccupancy': {'@MaxAdults': '2', '@MaxChildren': '0'}}, {'Name': 'Deluxe Room', 'Description': 'Deluxe Room - (1 single bed) - size_room: 22.85', 'RoomOccupancy': {'@MaxAdults': '1', '@MaxChildren': '0'}}, {'Name': 'Deluxe Room', 'Description': 'Deluxe Room - (1 single bed) - size_room: 22.85', 'RoomOccupancy': {'@MaxAdults': '1', '@MaxChildren': '0'}}, {'Name': 'Deluxe Room', 'Description': 'Deluxe Room - (1 single bed) - size_room: 22.85', 'RoomOccupancy': {'@MaxAdults': '1', '@MaxChildren': '0'}}, {'Name': 'Deluxe Room', 'Description': 'Deluxe Room - (1 single bed) - size_room: 22.85', 'RoomOccupancy': {'@MaxAdults': '1', '@MaxChildren': '0'}}, {'Name': 'Deluxe Room', 'Description': 'Deluxe Room - (1 single bed) - size_room: 22.85', 'RoomOccupancy': {'@MaxAdults': '1', '@MaxChildren': '0'}}, {'Name': 'Deluxe Room', 'Description': 'Deluxe Room - (1 single bed) - size_room: 22.85', 'RoomOccupancy': {'@MaxAdults': '1', '@MaxChildren': '0'}}, {'Name': 'Deluxe Room', 'Description': 'Deluxe Room - (1 single bed) - size_room: 22.85', 'RoomOccupancy': {'@MaxAdults': '1', '@MaxChildren': '0'}}, {'Name': 'Deluxe Room', 'Description': 'Deluxe Room - (1 single bed) - size_room: 22.85', 'RoomOccupancy': {'@MaxAdults': '1', '@MaxChildren': '0'}}, {'Name': 'Deluxe Room', 'Description': 'Deluxe Room - (1 single bed) - size_room: 22.85', 'RoomOccupancy': {'@MaxAdults': '1', '@MaxChildren': '0'}}, {'Name': 'Deluxe Room', 'Description': 'Deluxe Room - (1 single bed) - size_room: 22.85', 'RoomOccupancy': {'@MaxAdults': '1', '@MaxChildren': '0'}}, {'Name': 'Deluxe Room', 'Description': 'Deluxe Room - (1 single bed) - size_room: 22.85', 'RoomOccupancy': {'@MaxAdults': '1', '@MaxChildren': '0'}}, {'Name': 'Deluxe Room', 'Description': 'Deluxe Room - (1 single bed) - size_room: 22.85', 'RoomOccupancy': {'@MaxAdults': '1', '@MaxChildren': '0'}}, {'Name': 'Deluxe Room', 'Description': 'Deluxe Room - (1 single bed) - size_room: 22.85', 'RoomOccupancy': {'@MaxAdults': '1', '@MaxChildren': '0'}}, {'Name': 'Deluxe Room', 'Description': 'Deluxe Room - (1 single bed) - size_room: 22.85', 'RoomOccupancy': {'@MaxAdults': '1', '@MaxChildren': '0'}}, {'Name': 'Deluxe Room', 'Description': 'Deluxe Room - (1 single bed) - size_room: 22.85', 'RoomOccupancy': {'@MaxAdults': '1', '@MaxChildren': '0'}}, {'Name': 'Deluxe Room', 'Description': 'Deluxe Room - (1 single bed) - size_room: 22.85', 'RoomOccupancy': {'@MaxAdults': '1', '@MaxChildren': '0'}}, {'Name': 'Deluxe Room', 'Description': 'Deluxe Room - (1 single bed) - size_room: 22.85', 'RoomOccupancy': {'@MaxAdults': '1', '@MaxChildren': '0'}}, {'Name': 'Deluxe Room', 'Description': 'Deluxe Room - (1 single bed) - size_room: 22.85', 'RoomOccupancy': {'@MaxAdults': '1', '@MaxChildren': '0'}}, {'Name': 'Deluxe Room', 'Description': 'Deluxe Room - (1 single bed) - size_room: 22.85', 'RoomOccupancy': {'@MaxAdults': '1', '@MaxChildren': '0'}}, {'Name': 'Deluxe Room', 'Description': 'Deluxe Room - (1 single bed) - size_room: 22.85', 'RoomOccupancy': {'@MaxAdults': '1', '@MaxChildren': '0'}}, {'Name': 'Deluxe Room', 'Description': 'Deluxe Room - (1 single bed) - size_room: 22.85', 'RoomOccupancy': {'@MaxAdults': '1', '@MaxChildren': '0'}}, {'Name': 'Deluxe Room', 'Description': 'Deluxe Room - (1 single bed) - size_room: 22.85', 'RoomOccupancy': {'@MaxAdults': '1', '@MaxChildren': '0'}}, {'Name': 'Deluxe Room', 'Description': 'Deluxe Room - (1 single bed) - size_room: 22.85', 'RoomOccupancy': {'@MaxAdults': '1', '@MaxChildren': '0'}}, {'Name': 'Deluxe Room', 'Description': 'Deluxe Room - (1 single bed) - size_room: 22.85', 'RoomOccupancy': {'@MaxAdults': '1', '@MaxChildren': '0'}}, {'Name': 'Deluxe Room', 'Description': 'Deluxe Room - (1 single bed) - size_room: 22.85', 'RoomOccupancy': {'@MaxAdults': '1', '@MaxChildren': '0'}}, {'Name': 'Deluxe Room', 'Description': 'Deluxe Room - (1 single bed) - size_room: 22.85', 'RoomOccupancy': {'@MaxAdults': '1', '@MaxChildren': '0'}}, {'Name': 'Deluxe Room', 'Description': 'Deluxe Room - (1 single bed) - size_room: 22.85', 'RoomOccupancy': {'@MaxAdults': '1', '@MaxChildren': '0'}}, {'Name': 'Deluxe Room', 'Description': 'Deluxe Room - (1 single bed) - size_room: 22.85', 'RoomOccupancy': {'@MaxAdults': '1', '@MaxChildren': '0'}}, {'Name': 'Deluxe Room', 'Description': 'Deluxe Room - (1 single bed) - size_room: 22.85', 'RoomOccupancy': {'@MaxAdults': '1', '@MaxChildren': '0'}}, {'Name': 'Deluxe Room', 'Description': 'Deluxe Room - (1 single bed) - size_room: 22.85', 'RoomOccupancy': {'@MaxAdults': '1', '@MaxChildren': '0'}}, {'Name': 'Deluxe Room', 'Description': 'Deluxe Room - (1 single bed) - size_room: 22.85', 'RoomOccupancy': {'@MaxAdults': '1', '@MaxChildren': '0'}}, {'Name': 'Deluxe Room', 'Description': 'Deluxe Room - (1 single bed) - size_room: 22.85', 'RoomOccupancy': {'@MaxAdults': '1', '@MaxChildren': '0'}}, {'Name': 'Deluxe Room', 'Description': 'Deluxe Room - (1 single bed) - size_room: 22.85', 'RoomOccupancy': {'@MaxAdults': '1', '@MaxChildren': '0'}}, {'Name': 'Deluxe Room', 'Description': 'Deluxe Room - (1 single bed) - size_room: 22.85', 'RoomOccupancy': {'@MaxAdults': '1', '@MaxChildren': '0'}}, {'Name': 'Deluxe Room', 'Description': 'Deluxe Room - (1 single bed) - size_room: 22.85', 'RoomOccupancy': {'@MaxAdults': '1', '@MaxChildren': '0'}}, {'Name': 'Deluxe Room', 'Description': 'Deluxe Room - (1 single bed) - size_room: 22.85', 'RoomOccupancy': {'@MaxAdults': '1', '@MaxChildren': '0'}}, {'Name': 'Deluxe Room', 'Description': 'Deluxe Room - (1 single bed) - size_room: 22.85', 'RoomOccupancy': {'@MaxAdults': '1', '@MaxChildren': '0'}}, {'Name': 'Deluxe Room', 'Description': 'Deluxe Room - (1 single bed) - size_room: 22.85', 'RoomOccupancy': {'@MaxAdults': '1', '@MaxChildren': '0'}}, {'Name': 'Deluxe Room', 'Description': 'Deluxe Room - (1 single bed) - size_room: 22.85', 'RoomOccupancy': {'@MaxAdults': '1', '@MaxChildren': '0'}}, {'Name': 'Deluxe Room', 'Description': 'Deluxe Room - (1 single bed) - size_room: 22.85', 'RoomOccupancy': {'@MaxAdults': '1', '@MaxChildren': '0'}}, {'Name': 'Deluxe Room', 'Description': 'Deluxe Room - (1 single bed) - size_room: 22.85', 'RoomOccupancy': {'@MaxAdults': '1', '@MaxChildren': '0'}}, {'Name': 'Deluxe Room', 'Description': 'Deluxe Room - (1 single bed) - size_room: 22.85', 'RoomOccupancy': {'@MaxAdults': '1', '@MaxChildren': '0'}}, {'Name': 'Deluxe Room', 'Description': 'Deluxe Room - (1 single bed) - size_room: 22.85', 'RoomOccupancy': {'@MaxAdults': '1', '@MaxChildren': '0'}}, {'Name': 'Deluxe Room', 'Description': 'Deluxe Room - (1 single bed) - size_room: 22.85', 'RoomOccupancy': {'@MaxAdults': '1', '@MaxChildren': '0'}}, {'Name': 'Deluxe Room', 'Description': 'Deluxe Room - (1 single bed) - size_room: 22.85', 'RoomOccupancy': {'@MaxAdults': '1', '@MaxChildren': '0'}}, {'Name': 'Deluxe Room', 'Description': 'Deluxe Room - (1 single bed) - size_room: 22.85', 'RoomOccupancy': {'@MaxAdults': '1', '@MaxChildren': '0'}}, {'Name': 'Deluxe Room', 'Description': 'Deluxe Room - (1 single bed) - size_room: 22.85', 'RoomOccupancy': {'@MaxAdults': '1', '@MaxChildren': '0'}}, {'Name': 'Deluxe Room', 'Description': 'Deluxe Room - (1 single bed) - size_room: 22.85', 'RoomOccupancy': {'@MaxAdults': '1', '@MaxChildren': '0'}}, {'Name': 'Deluxe Room', 'Description': 'Deluxe Room - (1 single bed) - size_room: 22.85', 'RoomOccupancy': {'@MaxAdults': '1', '@MaxChildren': '0'}}, {'Name': 'Deluxe Room', 'Description': 'Deluxe Room - (1 single bed) - size_room: 22.85', 'RoomOccupancy': {'@MaxAdults': '1', '@MaxChildren': '0'}}, {'Name': 'Deluxe Room', 'Description': 'Deluxe Room - (1 single bed) - size_room: 22.85', 'RoomOccupancy': {'@MaxAdults': '1', '@MaxChildren': '0'}}, {'Name': 'Deluxe Room', 'Description': 'Deluxe Room - (1 single bed) - size_room: 22.85', 'RoomOccupancy': {'@MaxAdults': '1', '@MaxChildren': '0'}}, {'Name': 'Deluxe Room', 'Description': 'Deluxe Room - (1 single bed) - size_room: 22.85', 'RoomOccupancy': {'@MaxAdults': '1', '@MaxChildren': '0'}}, {'Name': 'Deluxe Room', 'Description': 'Deluxe Room - (1 single bed) - size_room: 22.85', 'RoomOccupancy': {'@MaxAdults': '1', '@MaxChildren': '0'}}, {'Name': 'Deluxe Room', 'Description': 'Deluxe Room - (1 single bed) - size_room: 22.85', 'RoomOccupancy': {'@MaxAdults': '1', '@MaxChildren': '0'}}, {'Name': 'Deluxe Room', 'Description': 'Deluxe Room - (1 single bed) - size_room: 22.85', 'RoomOccupancy': {'@MaxAdults': '1', '@MaxChildren': '0'}}, {'Name': 'Deluxe Room', 'Description': 'Deluxe Room - (1 single bed) - size_room: 22.85', 'RoomOccupancy': {'@MaxAdults': '1', '@MaxChildren': '0'}}, {'Name': 'Deluxe Room', 'Description': 'Deluxe Room - (1 single bed) - size_room: 22.85', 'RoomOccupancy': {'@MaxAdults': '1', '@MaxChildren': '0'}}, {'Name': 'Deluxe Room', 'Description': 'Deluxe Room - (1 single bed) - size_room: 22.85', 'RoomOccupancy': {'@MaxAdults': '1', '@MaxChildren': '0'}}, {'Name': 'Deluxe Room', 'Description': 'Deluxe Room - (1 single bed) - size_room: 22.85', 'RoomOccupancy': {'@MaxAdults': '1', '@MaxChildren': '0'}}, {'Name': 'Deluxe Room', 'Description': 'Deluxe Room - (1 single bed) - size_room: 22.85', 'RoomOccupancy': {'@MaxAdults': '1', '@MaxChildren': '0'}}, {'Name': 'Deluxe Room', 'Description': 'Deluxe Room - (1 single bed) - size_room: 22.85', 'RoomOccupancy': {'@MaxAdults': '1', '@MaxChildren': '0'}}, {'Name': 'Deluxe Room', 'Description': 'Deluxe Room - (1 single bed) - size_room: 22.85', 'RoomOccupancy': {'@MaxAdults': '1', '@MaxChildren': '0'}}, {'Name': 'Deluxe Room', 'Description': 'Deluxe Room - (1 single bed) - size_room: 22.85', 'RoomOccupancy': {'@MaxAdults': '1', '@MaxChildren': '0'}}, {'Name': 'Deluxe Room', 'Description': 'Deluxe Room - (1 single bed) - size_room: 22.85', 'RoomOccupancy': {'@MaxAdults': '1', '@MaxChildren': '0'}}, {'Name': 'Deluxe Room', 'Description': 'Deluxe Room - (1 single bed) - size_room: 22.85', 'RoomOccupancy': {'@MaxAdults': '1', '@MaxChildren': '0'}}, {'Name': 'Deluxe Room', 'Description': 'Deluxe Room - (1 single bed) - size_room: 22.85', 'RoomOccupancy': {'@MaxAdults': '1', '@MaxChildren': '0'}}, {'Name': 'Deluxe Room', 'Description': 'Deluxe Room - (1 single bed) - size_room: 22.85', 'RoomOccupancy': {'@MaxAdults': '1', '@MaxChildren': '0'}}, {'Name': 'Deluxe Room', 'Description': 'Deluxe Room - (1 single bed) - size_room: 22.85', 'RoomOccupancy': {'@MaxAdults': '1', '@MaxChildren': '0'}}, {'Name': 'Deluxe Room', 'Description': 'Deluxe Room - (1 single bed) - size_room: 22.85', 'RoomOccupancy': {'@MaxAdults': '1', '@MaxChildren': '0'}}, {'Name': 'Deluxe Room', 'Description': 'Deluxe Room - (1 single bed) - size_room: 22.85', 'RoomOccupancy': {'@MaxAdults': '1', '@MaxChildren': '0'}}, {'Name': 'Deluxe Room', 'Description': 'Deluxe Room - (1 single bed) - size_room: 22.85', 'RoomOccupancy': {'@MaxAdults': '1', '@MaxChildren': '0'}}, {'Name': 'Deluxe Room', 'Description': 'Deluxe Room - (1 single bed) - size_room: 22.85', 'RoomOccupancy': {'@MaxAdults': '1', '@MaxChildren': '0'}}, {'Name': 'Deluxe Room', 'Description': 'Deluxe Room - (1 single bed) - size_room: 22.85', 'RoomOccupancy': {'@MaxAdults': '1', '@MaxChildren': '0'}}, {'Name': 'Deluxe Room', 'Description': 'Deluxe Room - (1 single bed) - size_room: 22.85', 'RoomOccupancy': {'@MaxAdults': '1', '@MaxChildren': '0'}}, {'Name': 'Deluxe Room', 'Description': 'Deluxe Room - (1 single bed) - size_room: 22.85', 'RoomOccupancy': {'@MaxAdults': '1', '@MaxChildren': '0'}}, {'Name': 'Deluxe Room', 'Description': 'Deluxe Room - (1 single bed) - size_room: 22.85', 'RoomOccupancy': {'@MaxAdults': '1', '@MaxChildren': '0'}}, {'Name': 'Deluxe Room', 'Description': 'Deluxe Room - (1 single bed) - size_room: 22.85', 'RoomOccupancy': {'@MaxAdults': '1', '@MaxChildren': '0'}}, {'Name': 'Deluxe Room', 'Description': 'Deluxe Room - (1 single bed) - size_room: 22.85', 'RoomOccupancy': {'@MaxAdults': '1', '@MaxChildren': '0'}}, {'Name': 'Deluxe Room', 'Description': 'Deluxe Room - (1 single bed) - size_room: 22.85', 'RoomOccupancy': {'@MaxAdults': '1', '@MaxChildren': '0'}}, {'Name': 'Deluxe Room', 'Description': 'Deluxe Room - (1 single bed) - size_room: 22.85', 'RoomOccupancy': {'@MaxAdults': '1', '@MaxChildren': '0'}}, {'Name': 'Deluxe Room', 'Description': 'Deluxe Room - (1 single bed) - size_room: 22.85', 'RoomOccupancy': {'@MaxAdults': '1', '@MaxChildren': '0'}}, {'Name': 'Deluxe Room', 'Description': 'Deluxe Room - (1 single bed) - size_room: 22.85', 'RoomOccupancy': {'@MaxAdults': '1', '@MaxChildren': '0'}}, {'Name': 'Deluxe Room', 'Description': 'Deluxe Room - (1 single bed) - size_room: 22.85', 'RoomOccupancy': {'@MaxAdults': '1', '@MaxChildren': '0'}}, {'Name': 'Deluxe Room', 'Description': 'Deluxe Room - (1 single bed) - size_room: 22.85', 'RoomOccupancy': {'@MaxAdults': '1', '@MaxChildren': '0'}}, {'Name': 'Deluxe Room', 'Description': 'Deluxe Room - (1 single bed) - size_room: 22.85', 'RoomOccupancy': {'@MaxAdults': '1', '@MaxChildren': '0'}}, {'Name': 'Deluxe Room', 'Description': 'Deluxe Room - (1 single bed) - size_room: 22.85', 'RoomOccupancy': {'@MaxAdults': '1', '@MaxChildren': '0'}}, {'Name': 'Deluxe Room', 'Description': 'Deluxe Room - (1 single bed) - size_room: 22.85', 'RoomOccupancy': {'@MaxAdults': '1', '@MaxChildren': '0'}}, {'Name': 'Deluxe Room', 'Description': 'Deluxe Room - (1 single bed) - size_room: 22.85', 'RoomOccupancy': {'@MaxAdults': '1', '@MaxChildren': '0'}}, {'Name': 'Deluxe Room', 'Description': 'Deluxe Room - (1 single bed) - size_room: 22.85', 'RoomOccupancy': {'@MaxAdults': '1', '@MaxChildren': '0'}}, {'Name': 'Deluxe Room', 'Description': 'Deluxe Room - (1 single bed) - size_room: 22.85', 'RoomOccupancy': {'@MaxAdults': '1', '@MaxChildren': '0'}}, {'Name': 'Deluxe Room', 'Description': 'Deluxe Room - (1 single bed) - size_room: 22.85', 'RoomOccupancy': {'@MaxAdults': '1', '@MaxChildren': '0'}}, {'Name': 'Deluxe Room', 'Description': 'Deluxe Room - (1 single bed) - size_room: 22.85', 'RoomOccupancy': {'@MaxAdults': '1', '@MaxChildren': '0'}}, {'Name': 'Deluxe Room', 'Description': 'Deluxe Room - (1 single bed) - size_room: 22.85', 'RoomOccupancy': {'@MaxAdults': '1', '@MaxChildren': '0'}}, {'Name': 'Deluxe Room', 'Description': 'Deluxe Room - (1 single bed) - size_room: 22.85', 'RoomOccupancy': {'@MaxAdults': '1', '@MaxChildren': '0'}}, {'Name': 'Deluxe Room', 'Description': 'Deluxe Room - (1 single bed) - size_room: 22.85', 'RoomOccupancy': {'@MaxAdults': '1', '@MaxChildren': '0'}}, {'Name': 'Deluxe Room', 'Description': 'Deluxe Room - (1 single bed) - size_room: 22.85', 'RoomOccupancy': {'@MaxAdults': '1', '@MaxChildren': '0'}}, {'Name': 'Deluxe Room', 'Description': 'Deluxe Room - (1 single bed) - size_room: 22.85', 'RoomOccupancy': {'@MaxAdults': '1', '@MaxChildren': '0'}}, {'Name': 'Deluxe Room', 'Description': 'Deluxe Room - (1 single bed) - size_room: 22.85', 'RoomOccupancy': {'@MaxAdults': '1', '@MaxChildren': '0'}}, {'Name': 'Deluxe Room', 'Description': 'Deluxe Room - (1 single bed) - size_room: 22.85', 'RoomOccupancy': {'@MaxAdults': '1', '@MaxChildren': '0'}}, {'Name': 'Deluxe Room', 'Description': 'Deluxe Room - (1 single bed) - size_room: 22.85', 'RoomOccupancy': {'@MaxAdults': '1', '@MaxChildren': '0'}}, {'Name': 'Deluxe Room', 'Description': 'Deluxe Room - (1 single bed) - size_room: 22.85', 'RoomOccupancy': {'@MaxAdults': '1', '@MaxChildren': '0'}}, {'Name': 'Deluxe Room', 'Description': 'Deluxe Room - (1 single bed) - size_room: 22.85', 'RoomOccupancy': {'@MaxAdults': '1', '@MaxChildren': '0'}}, {'Name': 'Deluxe Room', 'Description': 'Deluxe Room - (1 single bed) - size_room: 22.85', 'RoomOccupancy': {'@MaxAdults': '1', '@MaxChildren': '0'}}, {'Name': 'Deluxe Room', 'Description': 'Deluxe Room - (1 single bed) - size_room: 22.85', 'RoomOccupancy': {'@MaxAdults': '1', '@MaxChildren': '0'}}, {'Name': 'Deluxe Room', 'Description': 'Deluxe Room - (1 single bed) - size_room: 22.85', 'RoomOccupancy': {'@MaxAdults': '1', '@MaxChildren': '0'}}, {'Name': 'Deluxe Room', 'Description': 'Deluxe Room - (1 single bed) - size_room: 22.85', 'RoomOccupancy': {'@MaxAdults': '1', '@MaxChildren': '0'}}, {'Name': 'Deluxe Room', 'Description': 'Deluxe Room - (1 single bed) - size_room: 22.85', 'RoomOccupancy': {'@MaxAdults': '1', '@MaxChildren': '0'}}, {'Name': 'Deluxe Room', 'Description': 'Deluxe Room - (1 single bed) - size_room: 22.85', 'RoomOccupancy': {'@MaxAdults': '1', '@MaxChildren': '0'}}, {'Name': 'Deluxe Room', 'Description': 'Deluxe Room - (1 single bed) - size_room: 22.85', 'RoomOccupancy': {'@MaxAdults': '1', '@MaxChildren': '0'}}, {'Name': 'Deluxe Room', 'Description': 'Deluxe Room - (1 single bed) - size_room: 22.85', 'RoomOccupancy': {'@MaxAdults': '1', '@MaxChildren': '0'}}, {'Name': 'Deluxe Room', 'Description': 'Deluxe Room - (1 single bed) - size_room: 22.85', 'RoomOccupancy': {'@MaxAdults': '1', '@MaxChildren': '0'}}, {'Name': 'Deluxe Room', 'Description': 'Deluxe Room - (1 single bed) - size_room: 22.85', 'RoomOccupancy': {'@MaxAdults': '1', '@MaxChildren': '0'}}, {'Name': 'Deluxe Room', 'Description': 'Deluxe Room - (1 single bed) - size_room: 22.85', 'RoomOccupancy': {'@MaxAdults': '1', '@MaxChildren': '0'}}, {'Name': 'Deluxe Room', 'Description': 'Deluxe Room - (1 single bed) - size_room: 22.85', 'RoomOccupancy': {'@MaxAdults': '1', '@MaxChildren': '0'}}, {'Name': 'Deluxe Room', 'Description': 'Deluxe Room - (1 single bed) - size_room: 22.85', 'RoomOccupancy': {'@MaxAdults': '1', '@MaxChildren': '0'}}, {'Name': 'Deluxe Room', 'Description': 'Deluxe Room - (1 single bed) - size_room: 22.85', 'RoomOccupancy': {'@MaxAdults': '1', '@MaxChildren': '0'}}, {'Name': 'Deluxe Room', 'Description': 'Deluxe Room - (1 single bed) - size_room: 22.85', 'RoomOccupancy': {'@MaxAdults': '1', '@MaxChildren': '0'}}, {'Name': 'Deluxe Room', 'Description': 'Deluxe Room - (1 single bed) - size_room: 22.85', 'RoomOccupancy': {'@MaxAdults': '1', '@MaxChildren': '0'}}, {'Name': 'Deluxe Room', 'Description': 'Deluxe Room - (1 single bed) - size_room: 22.85', 'RoomOccupancy': {'@MaxAdults': '1', '@MaxChildren': '0'}}, {'Name': 'Deluxe Room', 'Description': 'Deluxe Room - (1 single bed) - size_room: 22.85', 'RoomOccupancy': {'@MaxAdults': '1', '@MaxChildren': '0'}}, {'Name': 'Deluxe Room', 'Description': 'Deluxe Room - (1 single bed) - size_room: 22.85', 'RoomOccupancy': {'@MaxAdults': '1', '@MaxChildren': '0'}}, {'Name': 'Deluxe Room', 'Description': 'Deluxe Room - (1 single bed) - size_room: 22.85', 'RoomOccupancy': {'@MaxAdults': '1', '@MaxChildren': '0'}}, {'Name': 'Deluxe Room', 'Description': 'Deluxe Room - (1 single bed) - size_room: 22.85', 'RoomOccupancy': {'@MaxAdults': '1', '@MaxChildren': '0'}}, {'Name': 'Deluxe Room', 'Description': 'Deluxe Room - (1 single bed) - size_room: 22.85', 'RoomOccupancy': {'@MaxAdults': '1', '@MaxChildren': '0'}}, {'Name': 'Deluxe Room', 'Description': 'Deluxe Room - (1 single bed) - size_room: 22.85', 'RoomOccupancy': {'@MaxAdults': '1', '@MaxChildren': '0'}}, {'Name': 'Deluxe Room', 'Description': 'Deluxe Room - (1 single bed) - size_room: 22.85', 'RoomOccupancy': {'@MaxAdults': '1', '@MaxChildren': '0'}}, {'Name': 'Deluxe Room', 'Description': 'Deluxe Room - (1 single bed) - size_room: 22.85', 'RoomOccupancy': {'@MaxAdults': '1', '@MaxChildren': '0'}}, {'Name': 'Deluxe Room', 'Description': 'Deluxe Room - (1 single bed) - size_room: 22.85', 'RoomOccupancy': {'@MaxAdults': '1', '@MaxChildren': '0'}}, {'Name': 'Deluxe Room', 'Description': 'Deluxe Room - (1 single bed) - size_room: 22.85', 'RoomOccupancy': {'@MaxAdults': '1', '@MaxChildren': '0'}}, {'Name': 'Deluxe Room', 'Description': 'Deluxe Room - (1 single bed) - size_room: 22.85', 'RoomOccupancy': {'@MaxAdults': '1', '@MaxChildren': '0'}}, {'Name': 'Deluxe Room', 'Description': 'Deluxe Room - (1 single bed) - size_room: 22.85', 'RoomOccupancy': {'@MaxAdults': '1', '@MaxChildren': '0'}}, {'Name': 'Deluxe Room', 'Description': 'Deluxe Room - (1 single bed) - size_room: 22.85', 'RoomOccupancy': {'@MaxAdults': '1', '@MaxChildren': '0'}}, {'Name': 'Deluxe Room', 'Description': 'Deluxe Room - (1 single bed) - size_room: 22.85', 'RoomOccupancy': {'@MaxAdults': '1', '@MaxChildren': '0'}}, {'Name': 'Deluxe Room', 'Description': 'Deluxe Room - (1 single bed) - size_room: 22.85', 'RoomOccupancy': {'@MaxAdults': '1', '@MaxChildren': '0'}}, {'Name': 'Deluxe Room', 'Description': 'Deluxe Room - (1 single bed) - size_room: 22.85', 'RoomOccupancy': {'@MaxAdults': '1', '@MaxChildren': '0'}}, {'Name': 'Deluxe Room', 'Description': 'Deluxe Room - (1 single bed) - size_room: 22.85', 'RoomOccupancy': {'@MaxAdults': '1', '@MaxChildren': '0'}}, {'Name': 'Deluxe Room', 'Description': 'Deluxe Room - (1 single bed) - size_room: 22.85', 'RoomOccupancy': {'@MaxAdults': '1', '@MaxChildren': '0'}}, {'Name': 'Deluxe Room', 'Description': 'Deluxe Room - (1 single bed) - size_room: 22.85', 'RoomOccupancy': {'@MaxAdults': '1', '@MaxChildren': '0'}}, {'Name': 'Deluxe Room', 'Description': 'Deluxe Room - (1 single bed) - size_room: 22.85', 'RoomOccupancy': {'@MaxAdults': '1', '@MaxChildren': '0'}}, {'Name': 'Deluxe Room', 'Description': 'Deluxe Room - (1 single bed) - size_room: 22.85', 'RoomOccupancy': {'@MaxAdults': '1', '@MaxChildren': '0'}}, {'Name': 'Deluxe Room', 'Description': 'Deluxe Room - (1 single bed) - size_room: 22.85', 'RoomOccupancy': {'@MaxAdults': '1', '@MaxChildren': '0'}}, {'Name': 'Deluxe Room', 'Description': 'Deluxe Room - (1 single bed) - size_room: 22.85', 'RoomOccupancy': {'@MaxAdults': '1', '@MaxChildren': '0'}}, {'Name': 'Deluxe Room', 'Description': 'Deluxe Room - (1 single bed) - size_room: 22.85', 'RoomOccupancy': {'@MaxAdults': '1', '@MaxChildren': '0'}}, {'Name': 'Deluxe Room', 'Description': 'Deluxe Room - (1 single bed) - size_room: 22.85', 'RoomOccupancy': {'@MaxAdults': '1', '@MaxChildren': '0'}}, {'Name': 'Deluxe Room', 'Description': 'Deluxe Room - (1 single bed) - size_room: 22.85', 'RoomOccupancy': {'@MaxAdults': '1', '@MaxChildren': '0'}}, {'Name': 'Deluxe Room', 'Description': 'Deluxe Room - (1 single bed) - size_room: 22.85', 'RoomOccupancy': {'@MaxAdults': '1', '@MaxChildren': '0'}}, {'Name': 'Deluxe Room', 'Description': 'Deluxe Room - (1 single bed) - size_room: 22.85', 'RoomOccupancy': {'@MaxAdults': '1', '@MaxChildren': '0'}}, {'Name': 'Deluxe Room', 'Description': 'Deluxe Room - (1 single bed) - size_room: 22.85', 'RoomOccupancy': {'@MaxAdults': '1', '@MaxChildren': '0'}}, {'Name': 'Deluxe Room', 'Description': 'Deluxe Room - (1 single bed) - size_room: 22.85', 'RoomOccupancy': {'@MaxAdults': '1', '@MaxChildren': '0'}}, {'Name': 'Deluxe Room', 'Description': 'Deluxe Room - (1 single bed) - size_room: 22.85', 'RoomOccupancy': {'@MaxAdults': '1', '@MaxChildren': '0'}}, {'Name': 'Deluxe Room', 'Description': 'Deluxe Room - (1 single bed) - size_room: 22.85', 'RoomOccupancy': {'@MaxAdults': '1', '@MaxChildren': '0'}}, {'Name': 'Deluxe Room', 'Description': 'Deluxe Room - (1 single bed) - size_room: 22.85', 'RoomOccupancy': {'@MaxAdults': '1', '@MaxChildren': '0'}}, {'Name': 'Deluxe Room', 'Description': 'Deluxe Room - (1 single bed) - size_room: 22.85', 'RoomOccupancy': {'@MaxAdults': '1', '@MaxChildren': '0'}}, {'Name': 'Deluxe Room', 'Description': 'Deluxe Room - (1 single bed) - size_room: 22.85', 'RoomOccupancy': {'@MaxAdults': '1', '@MaxChildren': '0'}}, {'Name': 'Deluxe Room', 'Description': 'Deluxe Room - (1 single bed) - size_room: 22.85', 'RoomOccupancy': {'@MaxAdults': '1', '@MaxChildren': '0'}}, {'Name': 'Deluxe Room', 'Description': 'Deluxe Room - (1 single bed) - size_room: 22.85', 'RoomOccupancy': {'@MaxAdults': '1', '@MaxChildren': '0'}}, {'Name': 'Deluxe Room', 'Description': 'Deluxe Room - (1 single bed) - size_room: 22.85', 'RoomOccupancy': {'@MaxAdults': '1', '@MaxChildren': '0'}}, {'Name': 'Deluxe Room', 'Description': 'Deluxe Room - (1 single bed) - size_room: 22.85', 'RoomOccupancy': {'@MaxAdults': '1', '@MaxChildren': '0'}}, {'Name': 'Deluxe Room', 'Description': 'Deluxe Room - (1 single bed) - size_room: 22.85', 'RoomOccupancy': {'@MaxAdults': '1', '@MaxChildren': '0'}}, {'Name': 'Deluxe Room', 'Description': 'Deluxe Room - (1 single bed) - size_room: 22.85', 'RoomOccupancy': {'@MaxAdults': '1', '@MaxChildren': '0'}}, {'Name': 'Deluxe Room', 'Description': 'Deluxe Room - (1 single bed) - size_room: 22.85', 'RoomOccupancy': {'@MaxAdults': '1', '@MaxChildren': '0'}}, {'Name': 'Deluxe Room', 'Description': 'Deluxe Room - (1 single bed) - size_room: 22.85', 'RoomOccupancy': {'@MaxAdults': '1', '@MaxChildren': '0'}}, {'Name': 'Deluxe Room', 'Description': 'Deluxe Room - (1 single bed) - size_room: 22.85', 'RoomOccupancy': {'@MaxAdults': '1', '@MaxChildren': '0'}}, {'Name': 'Deluxe Room', 'Description': 'Deluxe Room - (1 single bed) - size_room: 22.85', 'RoomOccupancy': {'@MaxAdults': '1', '@MaxChildren': '0'}}, {'Name': 'Deluxe Room', 'Description': 'Deluxe Room - (1 single bed) - size_room: 22.85', 'RoomOccupancy': {'@MaxAdults': '1', '@MaxChildren': '0'}}, {'Name': 'Deluxe Room', 'Description': 'Deluxe Room - (1 single bed) - size_room: 22.85', 'RoomOccupancy': {'@MaxAdults': '1', '@MaxChildren': '0'}}, {'Name': 'Deluxe Room', 'Description': 'Deluxe Room - (1 single bed) - size_room: 22.85', 'RoomOccupancy': {'@MaxAdults': '1', '@MaxChildren': '0'}}, {'Name': 'Deluxe Room', 'Description': 'Deluxe Room - (1 single bed) - size_room: 22.85', 'RoomOccupancy': {'@MaxAdults': '1', '@MaxChildren': '0'}}, {'Name': 'Deluxe Room', 'Description': 'Deluxe Room - (1 single bed) - size_room: 22.85', 'RoomOccupancy': {'@MaxAdults': '1', '@MaxChildren': '0'}}, {'Name': 'Deluxe Room', 'Description': 'Deluxe Room - (1 single bed) - size_room: 22.85', 'RoomOccupancy': {'@MaxAdults': '1', '@MaxChildren': '0'}}, {'Name': 'Deluxe Room', 'Description': 'Deluxe Room - (1 single bed) - size_room: 22.85', 'RoomOccupancy': {'@MaxAdults': '1', '@MaxChildren': '0'}}, {'Name': 'Deluxe Room', 'Description': 'Deluxe Room - (1 single bed) - size_room: 22.85', 'RoomOccupancy': {'@MaxAdults': '1', '@MaxChildren': '0'}}, {'Name': 'Deluxe Room', 'Description': 'Deluxe Room - (1 single bed) - size_room: 22.85', 'RoomOccupancy': {'@MaxAdults': '1', '@MaxChildren': '0'}}, {'Name': 'Deluxe Room', 'Description': 'Deluxe Room - (1 single bed) - size_room: 22.85', 'RoomOccupancy': {'@MaxAdults': '1', '@MaxChildren': '0'}}, {'Name': 'Deluxe Room', 'Description': 'Deluxe Room - (1 single bed) - size_room: 22.85', 'RoomOccupancy': {'@MaxAdults': '1', '@MaxChildren': '0'}}, {'Name': 'Deluxe Room', 'Description': 'Deluxe Room - (1 single bed) - size_room: 22.85', 'RoomOccupancy': {'@MaxAdults': '1', '@MaxChildren': '0'}}, {'Name': 'Deluxe Room', 'Description': 'Deluxe Room - (1 single bed) - size_room: 22.85', 'RoomOccupancy': {'@MaxAdults': '1', '@MaxChildren': '0'}}, {'Name': 'Deluxe Room', 'Description': 'Deluxe Room - (1 single bed) - size_room: 22.85', 'RoomOccupancy': {'@MaxAdults': '1', '@MaxChildren': '0'}}, {'Name': 'Deluxe Room', 'Description': 'Deluxe Room - (1 single bed) - size_room: 22.85', 'RoomOccupancy': {'@MaxAdults': '1', '@MaxChildren': '0'}}, {'Name': 'Deluxe Room', 'Description': 'Deluxe Room - (1 single bed) - size_room: 22.85', 'RoomOccupancy': {'@MaxAdults': '1', '@MaxChildren': '0'}}, {'Name': 'Deluxe Room', 'Description': 'Deluxe Room - (1 single bed) - size_room: 22.85', 'RoomOccupancy': {'@MaxAdults': '1', '@MaxChildren': '0'}}, {'Name': 'Deluxe Room', 'Description': 'Deluxe Room - (1 single bed) - size_room: 22.85', 'RoomOccupancy': {'@MaxAdults': '1', '@MaxChildren': '0'}}, {'Name': 'Deluxe Room', 'Description': 'Deluxe Room - (1 single bed) - size_room: 22.85', 'RoomOccupancy': {'@MaxAdults': '1', '@MaxChildren': '0'}}, {'Name': 'Deluxe Room', 'Description': 'Deluxe Room - (1 single bed) - size_room: 22.85', 'RoomOccupancy': {'@MaxAdults': '1', '@MaxChildren': '0'}}, {'Name': 'Deluxe Room', 'Description': 'Deluxe Room - (1 single bed) - size_room: 22.85', 'RoomOccupancy': {'@MaxAdults': '1', '@MaxChildren': '0'}}, {'Name': 'Deluxe Room', 'Description': 'Deluxe Room - (1 single bed) - size_room: 22.85', 'RoomOccupancy': {'@MaxAdults': '1', '@MaxChildren': '0'}}, {'Name': 'Deluxe Room', 'Description': 'Deluxe Room - (1 single bed) - size_room: 22.85', 'RoomOccupancy': {'@MaxAdults': '1', '@MaxChildren': '0'}}, {'Name': 'Deluxe Room', 'Description': 'Deluxe Room - (1 single bed) - size_room: 22.85', 'RoomOccupancy': {'@MaxAdults': '1', '@MaxChildren': '0'}}, {'Name': 'Deluxe Room', 'Description': 'Deluxe Room - (1 single bed) - size_room: 22.85', 'RoomOccupancy': {'@MaxAdults': '1', '@MaxChildren': '0'}}, {'Name': 'Deluxe Room', 'Description': 'Deluxe Room - (1 single bed) - size_room: 22.85', 'RoomOccupancy': {'@MaxAdults': '1', '@MaxChildren': '0'}}, {'Name': 'Deluxe Room', 'Description': 'Deluxe Room - (1 single bed) - size_room: 22.85', 'RoomOccupancy': {'@MaxAdults': '1', '@MaxChildren': '0'}}, {'Name': 'Deluxe Room', 'Description': 'Deluxe Room - (1 single bed) - size_room: 22.85', 'RoomOccupancy': {'@MaxAdults': '1', '@MaxChildren': '0'}}, {'Name': 'Deluxe Room', 'Description': 'Deluxe Room - (1 single bed) - size_room: 22.85', 'RoomOccupancy': {'@MaxAdults': '1', '@MaxChildren': '0'}}, {'Name': 'Deluxe Room', 'Description': 'Deluxe Room - (1 single bed) - size_room: 22.85', 'RoomOccupancy': {'@MaxAdults': '1', '@MaxChildren': '0'}}, {'Name': 'Deluxe Room', 'Description': 'Deluxe Room - (1 single bed) - size_room: 22.85', 'RoomOccupancy': {'@MaxAdults': '1', '@MaxChildren': '0'}}, {'Name': 'Deluxe Room', 'Description': 'Deluxe Room - (1 single bed) - size_room: 22.85', 'RoomOccupancy': {'@MaxAdults': '1', '@MaxChildren': '0'}}, {'Name': 'Deluxe Room', 'Description': 'Deluxe Room - (1 single bed) - size_room: 22.85', 'RoomOccupancy': {'@MaxAdults': '1', '@MaxChildren': '0'}}, {'Name': 'Deluxe Room', 'Description': 'Deluxe Room - (1 single bed) - size_room: 22.85', 'RoomOccupancy': {'@MaxAdults': '1', '@MaxChildren': '0'}}, {'Name': 'Deluxe Room', 'Description': 'Deluxe Room - (1 single bed) - size_room: 22.85', 'RoomOccupancy': {'@MaxAdults': '1', '@MaxChildren': '0'}}, {'Name': 'Deluxe Room', 'Description': 'Deluxe Room - (1 single bed) - size_room: 22.85', 'RoomOccupancy': {'@MaxAdults': '1', '@MaxChildren': '0'}}, {'Name': 'Deluxe Room', 'Description': 'Deluxe Room - (1 single bed) - size_room: 22.85', 'RoomOccupancy': {'@MaxAdults': '1', '@MaxChildren': '0'}}, {'Name': 'Deluxe Room', 'Description': 'Deluxe Room - (1 single bed) - size_room: 22.85', 'RoomOccupancy': {'@MaxAdults': '1', '@MaxChildren': '0'}}, {'Name': 'Deluxe Room', 'Description': 'Deluxe Room - (1 single bed) - size_room: 22.85', 'RoomOccupancy': {'@MaxAdults': '1', '@MaxChildren': '0'}}, {'Name': 'Deluxe Room', 'Description': 'Deluxe Room - (1 single bed) - size_room: 22.85', 'RoomOccupancy': {'@MaxAdults': '1', '@MaxChildren': '0'}}, {'Name': 'Deluxe Room', 'Description': 'Deluxe Room - (1 single bed) - size_room: 22.85', 'RoomOccupancy': {'@MaxAdults': '1', '@MaxChildren': '0'}}, {'Name': 'Deluxe Room', 'Description': 'Deluxe Room - (1 single bed) - size_room: 22.85', 'RoomOccupancy': {'@MaxAdults': '1', '@MaxChildren': '0'}}, {'Name': 'Deluxe Room', 'Description': 'Deluxe Room - (1 single bed) - size_room: 22.85', 'RoomOccupancy': {'@MaxAdults': '1', '@MaxChildren': '0'}}, {'Name': 'Deluxe Room', 'Description': 'Deluxe Room - (1 single bed) - size_room: 22.85', 'RoomOccupancy': {'@MaxAdults': '1', '@MaxChildren': '0'}}, {'Name': 'Deluxe Room', 'Description': 'Deluxe Room - (1 single bed) - size_room: 22.85', 'RoomOccupancy': {'@MaxAdults': '1', '@MaxChildren': '0'}}, {'Name': 'Deluxe Room', 'Description': 'Deluxe Room - (1 single bed) - size_room: 22.85', 'RoomOccupancy': {'@MaxAdults': '1', '@MaxChildren': '0'}}, {'Name': 'Deluxe Room', 'Description': 'Deluxe Room - (1 single bed) - size_room: 22.85', 'RoomOccupancy': {'@MaxAdults': '1', '@MaxChildren': '0'}}, {'Name': 'Deluxe Room', 'Description': 'Deluxe Room - (1 single bed) - size_room: 22.85', 'RoomOccupancy': {'@MaxAdults': '1', '@MaxChildren': '0'}}, {'Name': 'Deluxe Room', 'Description': 'Deluxe Room - (1 single bed) - size_room: 22.85', 'RoomOccupancy': {'@MaxAdults': '1', '@MaxChildren': '0'}}, {'Name': 'Deluxe Room', 'Description': 'Deluxe Room - (1 single bed) - size_room: 22.85', 'RoomOccupancy': {'@MaxAdults': '1', '@MaxChildren': '0'}}, {'Name': 'Deluxe Room', 'Description': 'Deluxe Room - (1 single bed) - size_room: 22.85', 'RoomOccupancy': {'@MaxAdults': '1', '@MaxChildren': '0'}}, {'Name': 'Deluxe Room', 'Description': 'Deluxe Room - (1 single bed) - size_room: 22.85', 'RoomOccupancy': {'@MaxAdults': '1', '@MaxChildren': '0'}}, {'Name': 'Deluxe Room', 'Description': 'Deluxe Room - (1 single bed) - size_room: 22.85', 'RoomOccupancy': {'@MaxAdults': '1', '@MaxChildren': '0'}}, {'Name': 'Deluxe Room', 'Description': 'Deluxe Room - (1 single bed) - size_room: 22.85', 'RoomOccupancy': {'@MaxAdults': '1', '@MaxChildren': '0'}}, {'Name': 'Deluxe Room', 'Description': 'Deluxe Room - (1 single bed) - size_room: 22.85', 'RoomOccupancy': {'@MaxAdults': '1', '@MaxChildren': '0'}}, {'Name': 'Deluxe Room', 'Description': 'Deluxe Room - (1 single bed) - size_room: 22.85', 'RoomOccupancy': {'@MaxAdults': '1', '@MaxChildren': '0'}}, {'Name': 'Deluxe Room', 'Description': 'Deluxe Room - (1 single bed) - size_room: 22.85', 'RoomOccupancy': {'@MaxAdults': '1', '@MaxChildren': '0'}}, {'Name': 'Deluxe Room', 'Description': 'Deluxe Room - (1 single bed) - size_room: 22.85', 'RoomOccupancy': {'@MaxAdults': '1', '@MaxChildren': '0'}}, {'Name': 'Deluxe Room', 'Description': 'Deluxe Room - (1 single bed) - size_room: 22.85', 'RoomOccupancy': {'@MaxAdults': '1', '@MaxChildren': '0'}}, {'Name': 'Deluxe Room', 'Description': 'Deluxe Room - (1 single bed) - size_room: 22.85', 'RoomOccupancy': {'@MaxAdults': '1', '@MaxChildren': '0'}}, {'Name': 'One Bedroom Suite (Exclusive)', 'Description': 'One Bedroom Suite (Exclusive) - (1 super king bed) - size_room: 0', 'RoomOccupancy': {'@MaxAdults': '2', '@MaxChildren': '0'}}, {'Name': 'One Bedroom Suite (Exclusive)', 'Description': 'One Bedroom Suite (Exclusive) - (1 super king bed) - size_room: 0', 'RoomOccupancy': {'@MaxAdults': '2', '@MaxChildren': '0'}}, {'Name': 'One Bedroom Suite (Exclusive)', 'Description': 'One Bedroom Suite (Exclusive) - (1 super king bed) - size_room: 0', 'RoomOccupancy': {'@MaxAdults': '2', '@MaxChildren': '0'}}, {'Name': 'One Bedroom Suite (Exclusive)', 'Description': 'One Bedroom Suite (Exclusive) - (1 super king bed) - size_room: 0', 'RoomOccupancy': {'@MaxAdults': '2', '@MaxChildren': '0'}}, {'Name': 'One Bedroom Suite (Exclusive)', 'Description': 'One Bedroom Suite (Exclusive) - (1 super king bed) - size_room: 0', 'RoomOccupancy': {'@MaxAdults': '2', '@MaxChildren': '0'}}, {'Name': 'One Bedroom Suite (Exclusive)', 'Description': 'One Bedroom Suite (Exclusive) - (1 super king bed) - size_room: 0', 'RoomOccupancy': {'@MaxAdults': '2', '@MaxChildren': '0'}}, {'Name': 'One Bedroom Suite (Exclusive)', 'Description': 'One Bedroom Suite (Exclusive) - (1 super king bed) - size_room: 0', 'RoomOccupancy': {'@MaxAdults': '2', '@MaxChildren': '0'}}, {'Name': 'One Bedroom Suite (Exclusive)', 'Description': 'One Bedroom Suite (Exclusive) - (1 super king bed) - size_room: 0', 'RoomOccupancy': {'@MaxAdults': '2', '@MaxChildren': '0'}}, {'Name': 'One Bedroom Suite (Exclusive)', 'Description': 'One Bedroom Suite (Exclusive) - (1 super king bed) - size_room: 0', 'RoomOccupancy': {'@MaxAdults': '2', '@MaxChildren': '0'}}, {'Name': 'One Bedroom Suite (Exclusive)', 'Description': 'One Bedroom Suite (Exclusive) - (1 super king bed) - size_room: 0', 'RoomOccupancy': {'@MaxAdults': '2', '@MaxChildren': '0'}}, {'Name': 'One Bedroom Suite (Exclusive)', 'Description': 'One Bedroom Suite (Exclusive) - (1 super king bed) - size_room: 0', 'RoomOccupancy': {'@MaxAdults': '2', '@MaxChildren': '0'}}, {'Name': 'One Bedroom Suite (Exclusive)', 'Description': 'One Bedroom Suite (Exclusive) - (1 super king bed) - size_room: 0', 'RoomOccupancy': {'@MaxAdults': '2', '@MaxChildren': '0'}}, {'Name': 'One Bedroom Suite (Exclusive)', 'Description': 'One Bedroom Suite (Exclusive) - (1 super king bed) - size_room: 0', 'RoomOccupancy': {'@MaxAdults': '2', '@MaxChildren': '0'}}, {'Name': 'One Bedroom Suite (Exclusive)', 'Description': 'One Bedroom Suite (Exclusive) - (1 super king bed) - size_room: 0', 'RoomOccupancy': {'@MaxAdults': '2', '@MaxChildren': '0'}}, {'Name': 'One Bedroom Suite (Exclusive)', 'Description': 'One Bedroom Suite (Exclusive) - (1 super king bed) - size_room: 0', 'RoomOccupancy': {'@MaxAdults': '2', '@MaxChildren': '0'}}, {'Name': 'One Bedroom Suite (Exclusive)', 'Description': 'One Bedroom Suite (Exclusive) - (1 super king bed) - size_room: 0', 'RoomOccupancy': {'@MaxAdults': '2', '@MaxChildren': '0'}}, {'Name': 'One Bedroom Suite (Exclusive)', 'Description': 'One Bedroom Suite (Exclusive) - (1 super king bed) - size_room: 0', 'RoomOccupancy': {'@MaxAdults': '2', '@MaxChildren': '0'}}, {'Name': 'One Bedroom Suite (Exclusive)', 'Description': 'One Bedroom Suite (Exclusive) - (1 super king bed) - size_room: 0', 'RoomOccupancy': {'@MaxAdults': '2', '@MaxChildren': '0'}}, {'Name': 'One Bedroom Suite (Exclusive)', 'Description': 'One Bedroom Suite (Exclusive) - (1 super king bed) - size_room: 0', 'RoomOccupancy': {'@MaxAdults': '2', '@MaxChildren': '0'}}, {'Name': 'One Bedroom Suite (Exclusive)', 'Description': 'One Bedroom Suite (Exclusive) - (1 super king bed) - size_room: 0', 'RoomOccupancy': {'@MaxAdults': '2', '@MaxChildren': '0'}}, {'Name': 'One Bedroom Suite (Exclusive)', 'Description': 'One Bedroom Suite (Exclusive) - (1 super king bed) - size_room: 0', 'RoomOccupancy': {'@MaxAdults': '2', '@MaxChildren': '0'}}, {'Name': 'One Bedroom Suite (Exclusive)', 'Description': 'One Bedroom Suite (Exclusive) - (1 super king bed) - size_room: 0', 'RoomOccupancy': {'@MaxAdults': '2', '@MaxChildren': '0'}}, {'Name': 'One Bedroom Suite (Exclusive)', 'Description': 'One Bedroom Suite (Exclusive) - (1 super king bed) - size_room: 0', 'RoomOccupancy': {'@MaxAdults': '2', '@MaxChildren': '0'}}, {'Name': 'One Bedroom Suite (Exclusive)', 'Description': 'One Bedroom Suite (Exclusive) - (1 super king bed) - size_room: 0', 'RoomOccupancy': {'@MaxAdults': '2', '@MaxChildren': '0'}}, {'Name': 'One Bedroom Suite (Exclusive)', 'Description': 'One Bedroom Suite (Exclusive) - (1 super king bed) - size_room: 0', 'RoomOccupancy': {'@MaxAdults': '2', '@MaxChildren': '0'}}, {'Name': 'One Bedroom Suite (Exclusive)', 'Description': 'One Bedroom Suite (Exclusive) - (1 super king bed) - size_room: 0', 'RoomOccupancy': {'@MaxAdults': '2', '@MaxChildren': '0'}}, {'Name': 'One Bedroom Suite (Exclusive)', 'Description': 'One Bedroom Suite (Exclusive) - (1 super king bed) - size_room: 0', 'RoomOccupancy': {'@MaxAdults': '2', '@MaxChildren': '0'}}, {'Name': 'One Bedroom Suite (Exclusive)', 'Description': 'One Bedroom Suite (Exclusive) - (1 super king bed) - size_room: 0', 'RoomOccupancy': {'@MaxAdults': '2', '@MaxChildren': '0'}}, {'Name': 'One Bedroom Suite (Exclusive)', 'Description': 'One Bedroom Suite (Exclusive) - (1 super king bed) - size_room: 0', 'RoomOccupancy': {'@MaxAdults': '2', '@MaxChildren': '0'}}, {'Name': 'Executive Twin', 'Description': 'Executive Twin - (2 single beds) - size_room: 23', 'RoomOccupancy': {'@MaxAdults': '2', '@MaxChildren': '0'}}, {'Name': 'Executive Twin', 'Description': 'Executive Twin - (2 single beds) - size_room: 23', 'RoomOccupancy': {'@MaxAdults': '2', '@MaxChildren': '0'}}, {'Name': 'Executive Twin', 'Description': 'Executive Twin - (2 single beds) - size_room: 23', 'RoomOccupancy': {'@MaxAdults': '2', '@MaxChildren': '0'}}, {'Name': 'Executive Twin', 'Description': 'Executive Twin - (2 single beds) - size_room: 23', 'RoomOccupancy': {'@MaxAdults': '2', '@MaxChildren': '0'}}, {'Name': 'Executive Twin', 'Description': 'Executive Twin - (2 single beds) - size_room: 23', 'RoomOccupancy': {'@MaxAdults': '2', '@MaxChildren': '0'}}, {'Name': 'Executive Twin', 'Description': 'Executive Twin - (2 single beds) - size_room: 23', 'RoomOccupancy': {'@MaxAdults': '2', '@MaxChildren': '0'}}, {'Name': 'Executive Twin', 'Description': 'Executive Twin - (2 single beds) - size_room: 23', 'RoomOccupancy': {'@MaxAdults': '2', '@MaxChildren': '0'}}, {'Name': 'Executive Twin', 'Description': 'Executive Twin - (2 single beds) - size_room: 23', 'RoomOccupancy': {'@MaxAdults': '2', '@MaxChildren': '0'}}, {'Name': 'Executive Twin', 'Description': 'Executive Twin - (2 single beds) - size_room: 23', 'RoomOccupancy': {'@MaxAdults': '2', '@MaxChildren': '0'}}, {'Name': 'Executive Twin', 'Description': 'Executive Twin - (2 single beds) - size_room: 23', 'RoomOccupancy': {'@MaxAdults': '2', '@MaxChildren': '0'}}, {'Name': 'Executive Twin', 'Description': 'Executive Twin - (2 single beds) - size_room: 23', 'RoomOccupancy': {'@MaxAdults': '2', '@MaxChildren': '0'}}, {'Name': 'Executive Twin', 'Description': 'Executive Twin - (2 single beds) - size_room: 23', 'RoomOccupancy': {'@MaxAdults': '2', '@MaxChildren': '0'}}, {'Name': 'Executive Twin', 'Description': 'Executive Twin - (2 single beds) - size_room: 23', 'RoomOccupancy': {'@MaxAdults': '2', '@MaxChildren': '0'}}, {'Name': 'Executive Twin', 'Description': 'Executive Twin - (2 single beds) - size_room: 23', 'RoomOccupancy': {'@MaxAdults': '2', '@MaxChildren': '0'}}, {'Name': 'Executive Twin', 'Description': 'Executive Twin - (2 single beds) - size_room: 23', 'RoomOccupancy': {'@MaxAdults': '2', '@MaxChildren': '0'}}, {'Name': 'Executive Twin', 'Description': 'Executive Twin - (2 single beds) - size_room: 23', 'RoomOccupancy': {'@MaxAdults': '2', '@MaxChildren': '0'}}, {'Name': 'Executive Twin', 'Description': 'Executive Twin - (2 single beds) - size_room: 23', 'RoomOccupancy': {'@MaxAdults': '2', '@MaxChildren': '0'}}, {'Name': 'Executive Twin', 'Description': 'Executive Twin - (2 single beds) - size_room: 23', 'RoomOccupancy': {'@MaxAdults': '2', '@MaxChildren': '0'}}, {'Name': 'Executive Twin', 'Description': 'Executive Twin - (2 single beds) - size_room: 23', 'RoomOccupancy': {'@MaxAdults': '2', '@MaxChildren': '0'}}, {'Name': 'Executive Twin', 'Description': 'Executive Twin - (2 single beds) - size_room: 23', 'RoomOccupancy': {'@MaxAdults': '2', '@MaxChildren': '0'}}, {'Name': 'Executive Twin', 'Description': 'Executive Twin - (2 single beds) - size_room: 23', 'RoomOccupancy': {'@MaxAdults': '2', '@MaxChildren': '0'}}, {'Name': 'Executive Twin', 'Description': 'Executive Twin - (2 single beds) - size_room: 23', 'RoomOccupancy': {'@MaxAdults': '2', '@MaxChildren': '0'}}, {'Name': 'Executive Twin', 'Description': 'Executive Twin - (2 single beds) - size_room: 23', 'RoomOccupancy': {'@MaxAdults': '2', '@MaxChildren': '0'}}, {'Name': 'Executive Twin', 'Description': 'Executive Twin - (2 single beds) - size_room: 23', 'RoomOccupancy': {'@MaxAdults': '2', '@MaxChildren': '0'}}, {'Name': 'Executive Twin', 'Description': 'Executive Twin - (2 single beds) - size_room: 23', 'RoomOccupancy': {'@MaxAdults': '2', '@MaxChildren': '0'}}, {'Name': 'Executive Twin', 'Description': 'Executive Twin - (2 single beds) - size_room: 23', 'RoomOccupancy': {'@MaxAdults': '2', '@MaxChildren': '0'}}, {'Name': 'Executive Twin', 'Description': 'Executive Twin - (2 single beds) - size_room: 23', 'RoomOccupancy': {'@MaxAdults': '2', '@MaxChildren': '0'}}, {'Name': 'Executive Twin', 'Description': 'Executive Twin - (2 single beds) - size_room: 23', 'RoomOccupancy': {'@MaxAdults': '2', '@MaxChildren': '0'}}, {'Name': 'Executive Twin', 'Description': 'Executive Twin - (2 single beds) - size_room: 23', 'RoomOccupancy': {'@MaxAdults': '2', '@MaxChildren': '0'}}, {'Name': 'Executive Twin', 'Description': 'Executive Twin - (2 single beds) - size_room: 23', 'RoomOccupancy': {'@MaxAdults': '2', '@MaxChildren': '0'}}, {'Name': 'Executive Twin', 'Description': 'Executive Twin - (2 single beds) - size_room: 23', 'RoomOccupancy': {'@MaxAdults': '2', '@MaxChildren': '0'}}, {'Name': 'Executive Twin', 'Description': 'Executive Twin - (2 single beds) - size_room: 23', 'RoomOccupancy': {'@MaxAdults': '2', '@MaxChildren': '0'}}, {'Name': 'Executive Twin', 'Description': 'Executive Twin - (2 single beds) - size_room: 23', 'RoomOccupancy': {'@MaxAdults': '2', '@MaxChildren': '0'}}, {'Name': 'Executive Twin', 'Description': 'Executive Twin - (2 single beds) - size_room: 23', 'RoomOccupancy': {'@MaxAdults': '2', '@MaxChildren': '0'}}, {'Name': 'Executive Twin', 'Description': 'Executive Twin - (2 single beds) - size_room: 23', 'RoomOccupancy': {'@MaxAdults': '2', '@MaxChildren': '0'}}, {'Name': 'Executive Twin', 'Description': 'Executive Twin - (2 single beds) - size_room: 23', 'RoomOccupancy': {'@MaxAdults': '2', '@MaxChildren': '0'}}, {'Name': 'Executive Twin', 'Description': 'Executive Twin - (2 single beds) - size_room: 23', 'RoomOccupancy': {'@MaxAdults': '2', '@MaxChildren': '0'}}, {'Name': 'Executive Twin', 'Description': 'Executive Twin - (2 single beds) - size_room: 23', 'RoomOccupancy': {'@MaxAdults': '2', '@MaxChildren': '0'}}, {'Name': 'Executive Twin', 'Description': 'Executive Twin - (2 single beds) - size_room: 23', 'RoomOccupancy': {'@MaxAdults': '2', '@MaxChildren': '0'}}, {'Name': 'Executive Twin', 'Description': 'Executive Twin - (2 single beds) - size_room: 23', 'RoomOccupancy': {'@MaxAdults': '2', '@MaxChildren': '0'}}, {'Name': 'Executive Twin', 'Description': 'Executive Twin - (2 single beds) - size_room: 23', 'RoomOccupancy': {'@MaxAdults': '2', '@MaxChildren': '0'}}, {'Name': 'Executive Twin', 'Description': 'Executive Twin - (2 single beds) - size_room: 23', 'RoomOccupancy': {'@MaxAdults': '2', '@MaxChildren': '0'}}, {'Name': 'Executive Twin', 'Description': 'Executive Twin - (2 single beds) - size_room: 23', 'RoomOccupancy': {'@MaxAdults': '2', '@MaxChildren': '0'}}, {'Name': 'Executive Twin', 'Description': 'Executive Twin - (2 single beds) - size_room: 23', 'RoomOccupancy': {'@MaxAdults': '2', '@MaxChildren': '0'}}, {'Name': 'Executive Twin', 'Description': 'Executive Twin - (2 single beds) - size_room: 23', 'RoomOccupancy': {'@MaxAdults': '2', '@MaxChildren': '0'}}, {'Name': 'Executive Twin', 'Description': 'Executive Twin - (2 single beds) - size_room: 23', 'RoomOccupancy': {'@MaxAdults': '2', '@MaxChildren': '0'}}, {'Name': 'Executive Twin', 'Description': 'Executive Twin - (2 single beds) - size_room: 23', 'RoomOccupancy': {'@MaxAdults': '2', '@MaxChildren': '0'}}, {'Name': 'Executive Twin', 'Description': 'Executive Twin - (2 single beds) - size_room: 23', 'RoomOccupancy': {'@MaxAdults': '2', '@MaxChildren': '0'}}, {'Name': 'Executive Twin', 'Description': 'Executive Twin - (2 single beds) - size_room: 23', 'RoomOccupancy': {'@MaxAdults': '2', '@MaxChildren': '0'}}, {'Name': 'Executive Twin', 'Description': 'Executive Twin - (2 single beds) - size_room: 23', 'RoomOccupancy': {'@MaxAdults': '2', '@MaxChildren': '0'}}, {'Name': 'Executive Twin', 'Description': 'Executive Twin - (2 single beds) - size_room: 23', 'RoomOccupancy': {'@MaxAdults': '2', '@MaxChildren': '0'}}, {'Name': 'Executive Twin', 'Description': 'Executive Twin - (2 single beds) - size_room: 23', 'RoomOccupancy': {'@MaxAdults': '2', '@MaxChildren': '0'}}, {'Name': 'Executive Twin', 'Description': 'Executive Twin - (2 single beds) - size_room: 23', 'RoomOccupancy': {'@MaxAdults': '2', '@MaxChildren': '0'}}, {'Name': 'Executive Twin', 'Description': 'Executive Twin - (2 single beds) - size_room: 23', 'RoomOccupancy': {'@MaxAdults': '2', '@MaxChildren': '0'}}, {'Name': 'Executive Twin', 'Description': 'Executive Twin - (2 single beds) - size_room: 23', 'RoomOccupancy': {'@MaxAdults': '2', '@MaxChildren': '0'}}, {'Name': 'Executive Twin', 'Description': 'Executive Twin - (2 single beds) - size_room: 23', 'RoomOccupancy': {'@MaxAdults': '2', '@MaxChildren': '0'}}, {'Name': 'Executive Twin', 'Description': 'Executive Twin - (2 single beds) - size_room: 23', 'RoomOccupancy': {'@MaxAdults': '2', '@MaxChildren': '0'}}, {'Name': 'Executive Twin', 'Description': 'Executive Twin - (2 single beds) - size_room: 23', 'RoomOccupancy': {'@MaxAdults': '2', '@MaxChildren': '0'}}, {'Name': 'Executive Twin', 'Description': 'Executive Twin - (2 single beds) - size_room: 23', 'RoomOccupancy': {'@MaxAdults': '2', '@MaxChildren': '0'}}, {'Name': 'Executive Twin', 'Description': 'Executive Twin - (2 single beds) - size_room: 23', 'RoomOccupancy': {'@MaxAdults': '2', '@MaxChildren': '0'}}, {'Name': 'Executive Twin', 'Description': 'Executive Twin - (2 single beds) - size_room: 23', 'RoomOccupancy': {'@MaxAdults': '2', '@MaxChildren': '0'}}, {'Name': 'Executive Twin', 'Description': 'Executive Twin - (2 single beds) - size_room: 23', 'RoomOccupancy': {'@MaxAdults': '2', '@MaxChildren': '0'}}, {'Name': 'Executive Twin', 'Description': 'Executive Twin - (2 single beds) - size_room: 23', 'RoomOccupancy': {'@MaxAdults': '2', '@MaxChildren': '0'}}, {'Name': 'Executive Twin', 'Description': 'Executive Twin - (2 single beds) - size_room: 23', 'RoomOccupancy': {'@MaxAdults': '2', '@MaxChildren': '0'}}, {'Name': 'Executive Twin', 'Description': 'Executive Twin - (2 single beds) - size_room: 23', 'RoomOccupancy': {'@MaxAdults': '2', '@MaxChildren': '0'}}, {'Name': 'Executive Twin', 'Description': 'Executive Twin - (2 single beds) - size_room: 23', 'RoomOccupancy': {'@MaxAdults': '2', '@MaxChildren': '0'}}, {'Name': 'Executive Twin', 'Description': 'Executive Twin - (2 single beds) - size_room: 23', 'RoomOccupancy': {'@MaxAdults': '2', '@MaxChildren': '0'}}, {'Name': 'Executive Twin', 'Description': 'Executive Twin - (2 single beds) - size_room: 23', 'RoomOccupancy': {'@MaxAdults': '2', '@MaxChildren': '0'}}, {'Name': 'Executive Twin', 'Description': 'Executive Twin - (2 single beds) - size_room: 23', 'RoomOccupancy': {'@MaxAdults': '2', '@MaxChildren': '0'}}, {'Name': 'Executive Twin', 'Description': 'Executive Twin - (2 single beds) - size_room: 23', 'RoomOccupancy': {'@MaxAdults': '2', '@MaxChildren': '0'}}, {'Name': 'Executive Twin', 'Description': 'Executive Twin - (2 single beds) - size_room: 23', 'RoomOccupancy': {'@MaxAdults': '2', '@MaxChildren': '0'}}, {'Name': 'Executive Twin', 'Description': 'Executive Twin - (2 single beds) - size_room: 23', 'RoomOccupancy': {'@MaxAdults': '2', '@MaxChildren': '0'}}, {'Name': 'Executive Twin', 'Description': 'Executive Twin - (2 single beds) - size_room: 23', 'RoomOccupancy': {'@MaxAdults': '2', '@MaxChildren': '0'}}, {'Name': 'Executive Twin', 'Description': 'Executive Twin - (2 single beds) - size_room: 23', 'RoomOccupancy': {'@MaxAdults': '2', '@MaxChildren': '0'}}, {'Name': 'Executive Twin', 'Description': 'Executive Twin - (2 single beds) - size_room: 23', 'RoomOccupancy': {'@MaxAdults': '2', '@MaxChildren': '0'}}, {'Name': 'Executive Twin', 'Description': 'Executive Twin - (2 single beds) - size_room: 23', 'RoomOccupancy': {'@MaxAdults': '2', '@MaxChildren': '0'}}, {'Name': 'Executive Twin', 'Description': 'Executive Twin - (2 single beds) - size_room: 23', 'RoomOccupancy': {'@MaxAdults': '2', '@MaxChildren': '0'}}, {'Name': 'Executive Twin', 'Description': 'Executive Twin - (2 single beds) - size_room: 23', 'RoomOccupancy': {'@MaxAdults': '2', '@MaxChildren': '0'}}, {'Name': 'Executive Twin', 'Description': 'Executive Twin - (2 single beds) - size_room: 23', 'RoomOccupancy': {'@MaxAdults': '2', '@MaxChildren': '0'}}, {'Name': 'Executive Twin', 'Description': 'Executive Twin - (2 single beds) - size_room: 23', 'RoomOccupancy': {'@MaxAdults': '2', '@MaxChildren': '0'}}, {'Name': 'Executive Twin', 'Description': 'Executive Twin - (2 single beds) - size_room: 23', 'RoomOccupancy': {'@MaxAdults': '2', '@MaxChildren': '0'}}, {'Name': 'Executive Twin', 'Description': 'Executive Twin - (2 single beds) - size_room: 23', 'RoomOccupancy': {'@MaxAdults': '2', '@MaxChildren': '0'}}, {'Name': 'Executive Twin', 'Description': 'Executive Twin - (2 single beds) - size_room: 23', 'RoomOccupancy': {'@MaxAdults': '2', '@MaxChildren': '0'}}, {'Name': 'Executive Twin', 'Description': 'Executive Twin - (2 single beds) - size_room: 23', 'RoomOccupancy': {'@MaxAdults': '2', '@MaxChildren': '0'}}, {'Name': 'Executive Twin', 'Description': 'Executive Twin - (2 single beds) - size_room: 23', 'RoomOccupancy': {'@MaxAdults': '2', '@MaxChildren': '0'}}, {'Name': 'Executive Twin', 'Description': 'Executive Twin - (2 single beds) - size_room: 23', 'RoomOccupancy': {'@MaxAdults': '2', '@MaxChildren': '0'}}, {'Name': 'Executive Twin', 'Description': 'Executive Twin - (2 single beds) - size_room: 23', 'RoomOccupancy': {'@MaxAdults': '2', '@MaxChildren': '0'}}, {'Name': 'Executive Twin', 'Description': 'Executive Twin - (2 single beds) - size_room: 23', 'RoomOccupancy': {'@MaxAdults': '2', '@MaxChildren': '0'}}, {'Name': 'Executive Twin', 'Description': 'Executive Twin - (2 single beds) - size_room: 23', 'RoomOccupancy': {'@MaxAdults': '2', '@MaxChildren': '0'}}, {'Name': 'Executive Twin', 'Description': 'Executive Twin - (2 single beds) - size_room: 23', 'RoomOccupancy': {'@MaxAdults': '2', '@MaxChildren': '0'}}, {'Name': 'Executive Twin', 'Description': 'Executive Twin - (2 single beds) - size_room: 23', 'RoomOccupancy': {'@MaxAdults': '2', '@MaxChildren': '0'}}, {'Name': 'Executive Twin', 'Description': 'Executive Twin - (2 single beds) - size_room: 23', 'RoomOccupancy': {'@MaxAdults': '2', '@MaxChildren': '0'}}, {'Name': 'Executive Twin', 'Description': 'Executive Twin - (2 single beds) - size_room: 23', 'RoomOccupancy': {'@MaxAdults': '2', '@MaxChildren': '0'}}, {'Name': 'Executive Twin', 'Description': 'Executive Twin - (2 single beds) - size_room: 23', 'RoomOccupancy': {'@MaxAdults': '2', '@MaxChildren': '0'}}, {'Name': 'Executive Twin', 'Description': 'Executive Twin - (2 single beds) - size_room: 23', 'RoomOccupancy': {'@MaxAdults': '2', '@MaxChildren': '0'}}, {'Name': 'Executive Twin', 'Description': 'Executive Twin - (2 single beds) - size_room: 23', 'RoomOccupancy': {'@MaxAdults': '2', '@MaxChildren': '0'}}, {'Name': 'Executive Twin', 'Description': 'Executive Twin - (2 single beds) - size_room: 23', 'RoomOccupancy': {'@MaxAdults': '2', '@MaxChildren': '0'}}, {'Name': 'Executive Twin', 'Description': 'Executive Twin - (2 single beds) - size_room: 23', 'RoomOccupancy': {'@MaxAdults': '2', '@MaxChildren': '0'}}, {'Name': 'Executive Twin', 'Description': 'Executive Twin - (2 single beds) - size_room: 23', 'RoomOccupancy': {'@MaxAdults': '2', '@MaxChildren': '0'}}, {'Name': 'Executive Twin', 'Description': 'Executive Twin - (2 single beds) - size_room: 23', 'RoomOccupancy': {'@MaxAdults': '2', '@MaxChildren': '0'}}, {'Name': 'Executive Twin', 'Description': 'Executive Twin - (2 single beds) - size_room: 23', 'RoomOccupancy': {'@MaxAdults': '2', '@MaxChildren': '0'}}, {'Name': 'Executive Twin', 'Description': 'Executive Twin - (2 single beds) - size_room: 23', 'RoomOccupancy': {'@MaxAdults': '2', '@MaxChildren': '0'}}, {'Name': 'Executive Twin', 'Description': 'Executive Twin - (2 single beds) - size_room: 23', 'RoomOccupancy': {'@MaxAdults': '2', '@MaxChildren': '0'}}, {'Name': 'Executive Twin', 'Description': 'Executive Twin - (2 single beds) - size_room: 23', 'RoomOccupancy': {'@MaxAdults': '2', '@MaxChildren': '0'}}, {'Name': 'Executive Twin', 'Description': 'Executive Twin - (2 single beds) - size_room: 23', 'RoomOccupancy': {'@MaxAdults': '2', '@MaxChildren': '0'}}, {'Name': 'Executive Twin', 'Description': 'Executive Twin - (2 single beds) - size_room: 23', 'RoomOccupancy': {'@MaxAdults': '2', '@MaxChildren': '0'}}, {'Name': 'Room Assigned on Arrival', 'Description': '- (1 single bed) - size_room: 0', 'RoomOccupancy': {'@MaxAdults': '2', '@MaxChildren': '0'}}, {'Name': 'Room Assigned on Arrival', 'Description': '- (1 single bed) - size_room: 0', 'RoomOccupancy': {'@MaxAdults': '2', '@MaxChildren': '0'}}, {'Name': 'Room Assigned on Arrival', 'Description': '- (1 single bed) - size_room: 0', 'RoomOccupancy': {'@MaxAdults': '2', '@MaxChildren': '0'}}, {'Name': 'Room Assigned on Arrival', 'Description': '- (1 single bed) - size_room: 0', 'RoomOccupancy': {'@MaxAdults': '2', '@MaxChildren': '0'}}, {'Name': 'Room Assigned on Arrival', 'Description': '- (1 single bed) - size_room: 0', 'RoomOccupancy': {'@MaxAdults': '2', '@MaxChildren': '0'}}, {'Name': 'Room Assigned on Arrival', 'Description': '- (1 single bed) - size_room: 0', 'RoomOccupancy': {'@MaxAdults': '2', '@MaxChildren': '0'}}, {'Name': 'Room Assigned on Arrival', 'Description': '- (1 single bed) - size_room: 0', 'RoomOccupancy': {'@MaxAdults': '2', '@MaxChildren': '0'}}, {'Name': 'Room Assigned on Arrival', 'Description': '- (1 single bed) - size_room: 0', 'RoomOccupancy': {'@MaxAdults': '2', '@MaxChildren': '0'}}, {'Name': 'Room Assigned on Arrival', 'Description': '- (1 single bed) - size_room: 0', 'RoomOccupancy': {'@MaxAdults': '2', '@MaxChildren': '0'}}, {'Name': 'Room Assigned on Arrival', 'Description': '- (1 single bed) - size_room: 0', 'RoomOccupancy': {'@MaxAdults': '2', '@MaxChildren': '0'}}, {'Name': 'Room Assigned on Arrival', 'Description': '- (1 single bed) - size_room: 0', 'RoomOccupancy': {'@MaxAdults': '2', '@MaxChildren': '0'}}, {'Name': 'Room Assigned on Arrival', 'Description': '- (1 single bed) - size_room: 0', 'RoomOccupancy': {'@MaxAdults': '2', '@MaxChildren': '0'}}, {'Name': 'Room Assigned on Arrival', 'Description': '- (1 single bed) - size_room: 0', 'RoomOccupancy': {'@MaxAdults': '2', '@MaxChildren': '0'}}, {'Name': 'Room Assigned on Arrival', 'Description': '- (1 single bed) - size_room: 0', 'RoomOccupancy': {'@MaxAdults': '2', '@MaxChildren': '0'}}, {'Name': 'Room Assigned on Arrival', 'Description': '- (1 single bed) - size_room: 0', 'RoomOccupancy': {'@MaxAdults': '2', '@MaxChildren': '0'}}, {'Name': 'Room Assigned on Arrival', 'Description': '- (1 single bed) - size_room: 0', 'RoomOccupancy': {'@MaxAdults': '2', '@MaxChildren': '0'}}, {'Name': 'Room Assigned on Arrival', 'Description': '- (1 single bed) - size_room: 0', 'RoomOccupancy': {'@MaxAdults': '2', '@MaxChildren': '0'}}, {'Name': 'Room Assigned on Arrival', 'Description': '- (1 single bed) - size_room: 0', 'RoomOccupancy': {'@MaxAdults': '2', '@MaxChildren': '0'}}, {'Name': 'Room Assigned on Arrival', 'Description': '- (1 single bed) - size_room: 0', 'RoomOccupancy': {'@MaxAdults': '2', '@MaxChildren': '0'}}, {'Name': 'Room Assigned on Arrival', 'Description': '- (1 single bed) - size_room: 0', 'RoomOccupancy': {'@MaxAdults': '2', '@MaxChildren': '0'}}, {'Name': 'Room Assigned on Arrival', 'Description': '- (1 single bed) - size_room: 0', 'RoomOccupancy': {'@MaxAdults': '2', '@MaxChildren': '0'}}, {'Name': 'Room Assigned on Arrival', 'Description': '- (1 single bed) - size_room: 0', 'RoomOccupancy': {'@MaxAdults': '2', '@MaxChildren': '0'}}, {'Name': 'Room Assigned on Arrival', 'Description': '- (1 single bed) - size_room: 0', 'RoomOccupancy': {'@MaxAdults': '2', '@MaxChildren': '0'}}, {'Name': 'Room Assigned on Arrival', 'Description': '- (1 single bed) - size_room: 0', 'RoomOccupancy': {'@MaxAdults': '2', '@MaxChildren': '0'}}, {'Name': 'Room Assigned on Arrival', 'Description': '- (1 single bed) - size_room: 0', 'RoomOccupancy': {'@MaxAdults': '2', '@MaxChildren': '0'}}, {'Name': 'Room Assigned on Arrival', 'Description': '- (1 single bed) - size_room: 0', 'RoomOccupancy': {'@MaxAdults': '2', '@MaxChildren': '0'}}, {'Name': 'Luxury Suite', 'Description': 'Luxury Suite - (1 king bed) - size_room: 74', 'RoomOccupancy': {'@MaxAdults': '2', '@MaxChildren': '0'}}, {'Name': 'Luxury Suite', 'Description': 'Luxury Suite - (1 king bed) - size_room: 74', 'RoomOccupancy': {'@MaxAdults': '2', '@MaxChildren': '0'}}, {'Name': 'Luxury Suite', 'Description': 'Luxury Suite - (1 king bed) - size_room: 74', 'RoomOccupancy': {'@MaxAdults': '2', '@MaxChildren': '0'}}, {'Name': 'Luxury Suite', 'Description': 'Luxury Suite - (1 king bed) - size_room: 74', 'RoomOccupancy': {'@MaxAdults': '2', '@MaxChildren': '0'}}, {'Name': 'Luxury Suite', 'Description': 'Luxury Suite - (1 king bed) - size_room: 74', 'RoomOccupancy': {'@MaxAdults': '2', '@MaxChildren': '0'}}, {'Name': 'Luxury Suite', 'Description': 'Luxury Suite - (1 king bed) - size_room: 74', 'RoomOccupancy': {'@MaxAdults': '2', '@MaxChildren': '0'}}, {'Name': 'Luxury Suite', 'Description': 'Luxury Suite - (1 king bed) - size_room: 74', 'RoomOccupancy': {'@MaxAdults': '2', '@MaxChildren': '0'}}, {'Name': 'Luxury Suite', 'Description': 'Luxury Suite - (1 king bed) - size_room: 74', 'RoomOccupancy': {'@MaxAdults': '2', '@MaxChildren': '0'}}, {'Name': 'Luxury Suite', 'Description': 'Luxury Suite - (1 king bed) - size_room: 74', 'RoomOccupancy': {'@MaxAdults': '2', '@MaxChildren': '0'}}, {'Name': 'Luxury Suite', 'Description': 'Luxury Suite - (1 king bed) - size_room: 74', 'RoomOccupancy': {'@MaxAdults': '2', '@MaxChildren': '0'}}, {'Name': 'Luxury Suite', 'Description': 'Luxury Suite - (1 king bed) - size_room: 74', 'RoomOccupancy': {'@MaxAdults': '2', '@MaxChildren': '0'}}, {'Name': 'Luxury Suite', 'Description': 'Luxury Suite - (1 king bed) - size_room: 74', 'RoomOccupancy': {'@MaxAdults': '2', '@MaxChildren': '0'}}, {'Name': 'Luxury Suite', 'Description': 'Luxury Suite - (1 king bed) - size_room: 74', 'RoomOccupancy': {'@MaxAdults': '2', '@MaxChildren': '0'}}, {'Name': 'Luxury Suite', 'Description': 'Luxury Suite - (1 king bed) - size_room: 74', 'RoomOccupancy': {'@MaxAdults': '2', '@MaxChildren': '0'}}, {'Name': 'Luxury Suite', 'Description': 'Luxury Suite - (1 king bed) - size_room: 74', 'RoomOccupancy': {'@MaxAdults': '2', '@MaxChildren': '0'}}, {'Name': 'Luxury Suite', 'Description': 'Luxury Suite - (1 king bed) - size_room: 74', 'RoomOccupancy': {'@MaxAdults': '2', '@MaxChildren': '0'}}, {'Name': 'Luxury Suite', 'Description': 'Luxury Suite - (1 king bed) - size_room: 74', 'RoomOccupancy': {'@MaxAdults': '2', '@MaxChildren': '0'}}, {'Name': 'Luxury Suite', 'Description': 'Luxury Suite - (1 king bed) - size_room: 74', 'RoomOccupancy': {'@MaxAdults': '2', '@MaxChildren': '0'}}, {'Name': 'Luxury Suite', 'Description': 'Luxury Suite - (1 king bed) - size_room: 74', 'RoomOccupancy': {'@MaxAdults': '2', '@MaxChildren': '0'}}, {'Name': 'Superior Double Room', 'Description': 'Superior Double Room - (1 king bed) - size_room: 0', 'RoomOccupancy': {'@MaxAdults': '2', '@MaxChildren': '0'}}, {'Name': 'Superior Double Room', 'Description': 'Superior Double Room - (1 king bed) - size_room: 0', 'RoomOccupancy': {'@MaxAdults': '2', '@MaxChildren': '0'}}, {'Name': 'Superior Double Room', 'Description': 'Superior Double Room - (1 king bed) - size_room: 0', 'RoomOccupancy': {'@MaxAdults': '2', '@MaxChildren': '0'}}, {'Name': 'Superior Double Room', 'Description': 'Superior Double Room - (1 king bed) - size_room: 0', 'RoomOccupancy': {'@MaxAdults': '2', '@MaxChildren': '0'}}, {'Name': 'Superior Double Room', 'Description': 'Superior Double Room - (1 king bed) - size_room: 0', 'RoomOccupancy': {'@MaxAdults': '2', '@MaxChildren': '0'}}, {'Name': 'Superior Double Room', 'Description': 'Superior Double Room - (1 king bed) - size_room: 0', 'RoomOccupancy': {'@MaxAdults': '2', '@MaxChildren': '0'}}, {'Name': 'Superior Double Room', 'Description': 'Superior Double Room - (1 king bed) - size_room: 0', 'RoomOccupancy': {'@MaxAdults': '2', '@MaxChildren': '0'}}, {'Name': 'Superior Double Room', 'Description': 'Superior Double Room - (1 king bed) - size_room: 0', 'RoomOccupancy': {'@MaxAdults': '2', '@MaxChildren': '0'}}, {'Name': 'Superior Double Room', 'Description': 'Superior Double Room - (1 king bed) - size_room: 0', 'RoomOccupancy': {'@MaxAdults': '2', '@MaxChildren': '0'}}, {'Name': 'Superior Double Room', 'Description': 'Superior Double Room - (1 king bed) - size_room: 0', 'RoomOccupancy': {'@MaxAdults': '2', '@MaxChildren': '0'}}, {'Name': 'Superior Double Room', 'Description': 'Superior Double Room - (1 king bed) - size_room: 0', 'RoomOccupancy': {'@MaxAdults': '2', '@MaxChildren': '0'}}, {'Name': 'Superior Double Room', 'Description': 'Superior Double Room - (1 king bed) - size_room: 0', 'RoomOccupancy': {'@MaxAdults': '2', '@MaxChildren': '0'}}, {'Name': 'Superior Double Room', 'Description': 'Superior Double Room - (1 king bed) - size_room: 0', 'RoomOccupancy': {'@MaxAdults': '2', '@MaxChildren': '0'}}, {'Name': 'Superior Double Room', 'Description': 'Superior Double Room - (1 king bed) - size_room: 0', 'RoomOccupancy': {'@MaxAdults': '2', '@MaxChildren': '0'}}, {'Name': 'Superior Double Room', 'Description': 'Superior Double Room - (1 king bed) - size_room: 0', 'RoomOccupancy': {'@MaxAdults': '2', '@MaxChildren': '0'}}, {'Name': 'Superior Double Room', 'Description': 'Superior Double Room - (1 king bed) - size_room: 0', 'RoomOccupancy': {'@MaxAdults': '2', '@MaxChildren': '0'}}, {'Name': 'Superior Double Room', 'Description': 'Superior Double Room - (1 king bed) - size_room: 0', 'RoomOccupancy': {'@MaxAdults': '2', '@MaxChildren': '0'}}, {'Name': 'Superior Double Room', 'Description': 'Superior Double Room - (1 king bed) - size_room: 0', 'RoomOccupancy': {'@MaxAdults': '2', '@MaxChildren': '0'}}, {'Name': 'Superior Double Room', 'Description': 'Superior Double Room - (1 king bed) - size_room: 0', 'RoomOccupancy': {'@MaxAdults': '2', '@MaxChildren': '0'}}, {'Name': 'Superior Double Room', 'Description': 'Superior Double Room - (1 king bed) - size_room: 0', 'RoomOccupancy': {'@MaxAdults': '2', '@MaxChildren': '0'}}, {'Name': 'Superior Double Room', 'Description': 'Superior Double Room - (1 king bed) - size_room: 0', 'RoomOccupancy': {'@MaxAdults': '2', '@MaxChildren': '0'}}, {'Name': 'Superior Double Room', 'Description': 'Superior Double Room - (1 king bed) - size_room: 0', 'RoomOccupancy': {'@MaxAdults': '2', '@MaxChildren': '0'}}, {'Name': 'Superior Double Room', 'Description': 'Superior Double Room - (1 king bed) - size_room: 0', 'RoomOccupancy': {'@MaxAdults': '2', '@MaxChildren': '0'}}, {'Name': 'Superior Double Room', 'Description': 'Superior Double Room - (1 king bed) - size_room: 0', 'RoomOccupancy': {'@MaxAdults': '2', '@MaxChildren': '0'}}, {'Name': 'Superior Double Room', 'Description': 'Superior Double Room - (1 king bed) - size_room: 0', 'RoomOccupancy': {'@MaxAdults': '2', '@MaxChildren': '0'}}, {'Name': 'Superior Double Room', 'Description': 'Superior Double Room - (1 king bed) - size_room: 0', 'RoomOccupancy': {'@MaxAdults': '2', '@MaxChildren': '0'}}, {'Name': 'Superior Double Room', 'Description': 'Superior Double Room - (1 king bed) - size_room: 0', 'RoomOccupancy': {'@MaxAdults': '2', '@MaxChildren': '0'}}, {'Name': 'Superior Double Room', 'Description': 'Superior Double Room - (1 king bed) - size_room: 0', 'RoomOccupancy': {'@MaxAdults': '2', '@MaxChildren': '0'}}, {'Name': 'Superior Double Room', 'Description': 'Superior Double Room - (1 king bed) - size_room: 0', 'RoomOccupancy': {'@MaxAdults': '2', '@MaxChildren': '0'}}, {'Name': 'Superior Double Room', 'Description': 'Superior Double Room - (1 king bed) - size_room: 0', 'RoomOccupancy': {'@MaxAdults': '2', '@MaxChildren': '0'}}, {'Name': 'Superior Double Room', 'Description': 'Superior Double Room - (1 king bed) - size_room: 0', 'RoomOccupancy': {'@MaxAdults': '2', '@MaxChildren': '0'}}, {'Name': 'Superior Double Room', 'Description': 'Superior Double Room - (1 king bed) - size_room: 0', 'RoomOccupancy': {'@MaxAdults': '2', '@MaxChildren': '0'}}, {'Name': 'Superior Double Room', 'Description': 'Superior Double Room - (1 king bed) - size_room: 0', 'RoomOccupancy': {'@MaxAdults': '2', '@MaxChildren': '0'}}, {'Name': 'Superior Double Room', 'Description': 'Superior Double Room - (1 king bed) - size_room: 0', 'RoomOccupancy': {'@MaxAdults': '2', '@MaxChildren': '0'}}, {'Name': 'Superior Double Room', 'Description': 'Superior Double Room - (1 king bed) - size_room: 0', 'RoomOccupancy': {'@MaxAdults': '2', '@MaxChildren': '0'}}]}, 'JPRooms': {'JPRoom': [{'@JRCode': 'JR000015'}, {'@JRCode': 'JR000036'}, {'@JRCode': 'JR002526'}, {'@JRCode': 'JR000018'}, {'@JRCode': 'JR287035'}, {'@JRCode': 'JR002861'}, {'@JRCode': 'JR002598'}, {'@JRCode': 'JR312710'}, {'@JRCode': 'JR000720'}, {'@JRCode': 'JR299314'}, {'@JRCode': 'JR020525'}, {'@JRCode': 'JR000495'}, {'@JRCode': 'JR001806'}, {'@JRCode': 'JR001804'}, {'@JRCode': 'JR333961'}, {'@JRCode': 'JR333405'}, {'@JRCode': 'JR009832'}, {'@JRCode': 'JR001798'}, {'@JRCode': 'JR000721'}, {'@JRCode': 'JR000040'}, {'@JRCode': 'JR003065'}, {'@JRCode': 'JR000050'}, {'@JRCode': 'JR002938'}, {'@JRCode': 'JR005577'}, {'@JRCode': 'JR004831'}, {'@JRCode': 'JR33356250'}, {'@JRCode': 'JR000007'}, {'@JRCode': 'JR002358'}, {'@JRCode': 'JR000003'}, {'@JRCode': 'JR161290'}, {'@JRCode': 'JR000308'}, {'@JRCode': 'JR000486'}, {'@JRCode': 'JR011296'}, {'@JRCode': 'JR013011'}, {'@JRCode': 'JR369194'}, {'@JRCode': 'JR369160'}, {'@JRCode': 'JR000073'}, {'@JRCode': 'JR002264'}, {'@JRCode': 'JR008185'}, {'@JRCode': 'JR000072'}, {'@JRCode': 'JR389442'}, {'@JRCode': 'JR001833'}, {'@JRCode': 'JR389648'}, {'@JRCode': 'JR389186'}, {'@JRCode': 'JR015577'}, {'@JRCode': 'JR014332'}, {'@JRCode': 'JR000539'}, {'@JRCode': 'JR395660'}, {'@JRCode': 'JR022646'}, {'@JRCode': 'JR002797'}, {'@JRCode': 'JR032931'}, {'@JRCode': 'JR33356207'}, {'@JRCode': 'JR33358211'}, {'@JRCode': 'JR33356913'}, {'@JRCode': 'JR33357289'}, {'@JRCode': 'JR33362565'}, {'@JRCode': 'JR33357193'}, {'@JRCode': 'JR000320'}, {'@JRCode': 'JR000008'}, {'@JRCode': 'JR000051'}, {'@JRCode': 'JR000095'}, {'@JRCode': 'JR000593'}, {'@JRCode': 'JR000209'}, {'@JRCode': 'JR000462'}, {'@JRCode': 'JR001208'}, {'@JRCode': 'JR000027'}, {'@JRCode': 'JR000224'}]}, 'JPCodeSynonyms': {'JPCodeSynonym': {'@JPCode': 'JP128519'}}}}}}}}}